In [1]:
import json
import os
import argparse

import numpy as np
import random
import torch
from torch import nn, optim
from tqdm.auto import tqdm
from transformers import AutoImageProcessor, AutoModelForImageClassification, AutoConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from pathlib import Path
from torch.utils.data import DataLoader, Subset
import sys
sys.path.append('../../lib/exlib/src')
from exlib.modules.sop import SOPConfig, SOPTextCls, get_chained_attr, compress_masks_text, AttributionOutputSOP

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

SEED = 42
if SEED != -1:
    # Torch RNG
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    # Python RNG
    np.random.seed(SEED)
    random.seed(SEED)

In [3]:
# model paths
backbone_model_name = 'textattack/bert-base-uncased-SST-2'
backbone_processor_name = 'textattack/bert-base-uncased-SST-2'
# sop_config_path = 'configs/imagenet_m.json'

# data paths
# TRAIN_DATA_DIR = '../data/imagenet_m/train'
# VAL_DATA_DIR = '../data/imagenet_m/val'

# training args
batch_size = 1
lr = 0.000005
num_epochs = 20
warmup_steps = 2000
mask_batch_size = 4

# experiment args
exp_dir = '../../exps/sst_m_1h_gg2.0_gs1.0_ggta1.0_ggtb0.1/best'

In [4]:
backbone_model = AutoModelForSequenceClassification.from_pretrained(backbone_model_name)
processor = AutoTokenizer.from_pretrained(backbone_processor_name)
backbone_config = AutoConfig.from_pretrained(backbone_model_name)

config = SOPConfig(json_file=os.path.join(exp_dir, 'config.json'),
                   projected_input_scale=2)
print(config.__dict__)

{'hidden_size': 768, 'input_hidden_size': None, 'num_labels': 2, 'projected_input_scale': 2, 'num_heads': 1, 'num_masks_sample': 8, 'num_masks_max': 16, 'image_size': [224, 224], 'num_channels': 3, 'attn_patch_size': 16, 'finetune_layers': ['model.classifier'], 'group_gen_scale': 2.0, 'group_gen_temp_alpha': 1.0, 'group_gen_temp_beta': 0.1, 'group_sel_scale': 1.0, 'freeze_projection': False}


In [5]:
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
import jsonlines


class SSTDataset(Dataset):
    def __init__(self, data_path, data_size=-1, transform=None):
        self.data_path = data_path
        self.data_size = data_size
        self.transform = transform
        self.documents = []
        self.labels = []
        with jsonlines.open(data_path) as reader:
            for obj in reader:
                self.documents.append(obj['document'])
                self.labels.append(obj['label'])
        self.classes = sorted(set(self.labels))
        
        if data_size != -1:
            # select a subset of the data so that each class has data_size number of documents
            documents = []
            labels = []
            for c in self.classes:
                c_docs = [doc for doc, label in zip(self.documents, self.labels) if label == c]
                documents.extend(c_docs[:data_size])
                labels.extend([c]*data_size)
            self.documents = documents
            self.labels = labels

        assert len(self.documents) == len(self.labels)
        
        print(f'Loaded {len(self.labels)} documents of {len(self.classes)} classes')
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, idx):
        inputs = self.transform(self.documents[idx])
        inputs['label'] = self.labels[idx]
        return inputs

        
# Path to your dataset file
train_path = '../../data/SST/data/train.jsonl'
val_path = '../../data/SST/data/dev.jsonl'

# Tokenization function
def transform(batch):
    return processor(batch, 
                   # padding="max_length", 
                   truncation=True)
                   #   , 
                   # max_length=512)

# Load the dataset from the file
train_size, val_size = -1, -1 #100, 100
train_dataset = SSTDataset(train_path, data_size=train_size, transform=transform)
val_dataset = SSTDataset(val_path, data_size=val_size, transform=transform)

# Create a DataLoader to batch and shuffle the data
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Loaded 6920 documents of 2 classes
Loaded 872 documents of 2 classes


In [6]:
from collections import namedtuple

WrappedBackboneOutput = namedtuple("WrappedBackboneOutput", 
                                  ["logits",
                                   "pooler_output"])


class WrappedBackboneModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, inputs=None, **kwargs):
        outputs = self.model(inputs, output_hidden_states=True, **kwargs)
        return WrappedBackboneOutput(outputs.logits, outputs.hidden_states[-1][:,0])

In [7]:
class WrappedModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, **kwargs):
        outputs = self.model(**kwargs)
        return outputs.logits

In [8]:
class WrappedBackboneModel2(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, inputs=None, **kwargs):
        # device = next(self.model.parameters()).device
        # inputs = inputs.to(device)
        # kwargs = {k: v.to(device) for k, v in kwargs.items()}
        outputs = self.model(inputs, output_hidden_states=True, **kwargs)
        # import pdb; pdb.set_trace()
        return torch.softmax(outputs.logits, dim=-1)
original_model2 = WrappedBackboneModel2(backbone_model).to(device)

In [9]:
class WrappedBackboneModel3(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, inputs=None, **kwargs):
        # device = next(self.model.parameters()).device
        # inputs = inputs.to(device)
        # kwargs = {k: v.to(device) for k, v in kwargs.items()}
        outputs = self.model(inputs, output_hidden_states=True, **kwargs)
        # import pdb; pdb.set_trace()
        return outputs.logits
original_model3 = WrappedBackboneModel3(backbone_model).to(device)

In [10]:
wrapped_backbone_model = WrappedBackboneModel(backbone_model)
wrapped_backbone_model = wrapped_backbone_model.to(device)
class_weights = get_chained_attr(wrapped_backbone_model, config.finetune_layers[0]).weight #.clone().to(device)
projection_layer = wrapped_backbone_model.model.bert.embeddings.word_embeddings

In [11]:
model = SOPTextCls(config, wrapped_backbone_model, class_weights=class_weights, projection_layer=projection_layer)
state_dict = torch.load(os.path.join(exp_dir, 'checkpoint.pth'))
model.load_state_dict(state_dict['model'], strict=False)
model = model.to(device)
model.eval();

projection layer is not frozen


In [12]:
original_model = WrappedModel(backbone_model)
original_model = original_model.to(device)

In [13]:
batch = next(iter(val_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'label'])


In [14]:
if not isinstance(batch['input_ids'], torch.Tensor):
    inputs = torch.stack(batch['input_ids']).transpose(0, 1).to(device)
    if 'token_type_ids' in batch:
        token_type_ids = torch.stack(batch['token_type_ids']).transpose(0, 1).to(device)
    else:
        token_type_ids = None
    attention_mask = torch.stack(batch['attention_mask']).transpose(0, 1).to(device)

    # concatenated_rows = [torch.stack(sublist) for sublist in batch['segs']]
    # segs = torch.stack(concatenated_rows).permute(2, 0, 1).to(device).float()
    # print('segs', segs.shape)
else:
    inputs = batch['input_ids'].to(device)
    if 'token_type_ids' in batch:
        token_type_ids = batch['token_type_ids'].to(device)
    else:
        token_type_ids = None
    attention_mask = batch['attention_mask'].to(device)
    # segs = batch['segs'].to(device).float()
kwargs = {
    'token_type_ids': token_type_ids,
    'attention_mask': attention_mask,
}
labels = batch['label'].to(device)

## Archipelago

In [15]:
from exlib.explainers.archipelago import ArchipelagoTextCls

explainer = ArchipelagoTextCls(backbone_model).to(device)
arc_expln = explainer(inputs, labels, **kwargs)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [16]:
arc_expln.attributions.shape, arc_expln.explainer_output['expln_flat_masks'][0].shape

(torch.Size([1, 16]), torch.Size([16]))

In [17]:
j = 0
grouped_attrs = []
for i in tqdm(range(2)):
    expln_i = explainer(inputs, torch.tensor([i]), **kwargs)
    grouped_attrs.append(expln_i.explainer_output['mask_weights'][j])
grouped_attrs_aggr = torch.tensor([sum(ga) for ga in grouped_attrs]).to(device)
aggr_pred = torch.argmax(grouped_attrs_aggr)
aggr_pred, grouped_attrs_aggr

  0%|          | 0/2 [00:00<?, ?it/s]

(tensor(0, device='cuda:0'), tensor([ 2.1138, -2.9271], device='cuda:0'))

## Lime

In [28]:
from exlib.explainers.lime import LimeTextCls

eik = {
    "top_labels": 2, 
    "num_samples": 1000
}

def split_expression(x):
    # return processor.tokenize(x)
    # import pdb; pdb.set_trace()
    tokens = x.split()
    # tokens = [' ' + a for a in x.split()]
    # tokens[0] = tokens[0][1:]
    # tokens = processor.tokenize(x)
    # import pdb; pdb.set_trace()
    return tokens
    
ltek = {
    "mask_string": "[PAD]",
    "split_expression": split_expression,
    # "feature_selection": 'none'
}

lime_explainer = LimeTextCls(original_model, processor,
                             LimeTextExplainerKwargs=ltek,
                             explain_instance_kwargs=eik)

lime_expln = lime_explainer(inputs, labels)

In [29]:
lime_expln.explainer_output[0].local_exp

{0: [(4, -1.592656293443421),
  (7, -0.5441038290068904),
  (5, -0.27116549053330047),
  (9, 0.263618523919542),
  (12, -0.261171586233251),
  (3, -0.13977285823498853),
  (6, -0.1383914754140747),
  (10, -0.09634597021900294),
  (8, 0.0940371753952921),
  (1, -0.07112639941394419)],
 1: [(4, 1.6026213682368888),
  (7, 0.5123724939228225),
  (9, -0.2730581651722103),
  (5, 0.24266287354136043),
  (12, 0.2349818081003179),
  (3, 0.1249267494976049),
  (6, 0.09655836112529485),
  (1, 0.0888565240840873),
  (8, -0.08660559611596468),
  (10, 0.08242956527716251)]}

In [30]:
lime_expln.explainer_output[0].as_list()

[('lovely', 1.6026213682368888),
 ('performances', 0.5123724939228225),
 ('buy', -0.2730581651722103),
 ('film', 0.24266287354136043),
 ('##ors', 0.2349818081003179),
 ('a', 0.1249267494976049),
 ('with', 0.09655836112529485),
 ('it', 0.0888565240840873),
 ('by', -0.08660559611596468),
 ('and', 0.08242956527716251)]

In [31]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks):
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

In [32]:
tokens = processor.convert_ids_to_tokens(inputs[0])

In [33]:
printmd(show_attrs(tokens, lime_expln.attributions[0]))

 [CLS]<span style='background-color:rgb(232.34158594161272,22.65841405838728,22.65841405838728)'> it</span> s<span style='background-color:rgb(223.14367979764938,31.856320202350616,31.856320202350616)'> a</span><span style='background-color:rgb(-153.66843581199646,408.66843581199646,408.66843581199646)'> lovely</span><span style='background-color:rgb(193.12096640467644,61.87903359532356,61.87903359532356)'> film</span><span style='background-color:rgb(230.37761762738228,24.62238237261772,24.62238237261772)'> with</span><span style='background-color:rgb(-153.66843581199646,408.66843581199646,408.66843581199646)'> lovely</span><span style='background-color:rgb(124.3450140953064,130.6549859046936,130.6549859046936)'> performances</span><span style='background-color:rgb(277.08442635834217,-22.08442635834217,-22.08442635834217)'> by</span><span style='background-color:rgb(324.6298348903656,-69.6298348903656,-69.6298348903656)'> buy</span><span style='background-color:rgb(233.9804608002305,21.019539199769497,21.019539199769497)'> and</span> acc<span style='background-color:rgb(195.07963970303535,59.920360296964645,59.920360296964645)'> ##ors</span> ##i [SEP]

In [29]:
tokens1 = processor.convert_ids_to_tokens(inputs[1])
printmd(show_attrs(tokens1, lime_expln.attributions[1]))

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [20]:
lime_expln.explainer_output[0].as_list()

[('[PAD]', 0.5767190980600844),
 ('the', 0.31360959520663556),
 ('[SEP]', 0.24867316109376783),
 ('–', 0.19954130976062953),
 ('or', 0.16558237185584104),
 ('|', -0.1487878343240845),
 ('.', -0.1309357914639342),
 ('erase', -0.12731568318035072),
 ('allan', -0.12423464436928029),
 ('so', -0.07478245937713171)]

In [21]:
lime_expln.explainer_output[0].local_exp

{0: [(208, -0.7893760245796537),
  (198, -0.30193107376986145),
  (9, -0.30047980664344687),
  (141, -0.2145088359383795),
  (203, -0.21344021701543137),
  (96, -0.21216588209013354),
  (1, 0.13338063608778614),
  (207, 0.12451006101765646),
  (205, 0.11947097009523455),
  (178, 0.08199956342861209)],
 1: [(208, 0.5767190980600844),
  (9, 0.31360959520663556),
  (198, 0.24867316109376783),
  (141, 0.19954130976062953),
  (203, 0.16558237185584104),
  (205, -0.1487878343240845),
  (13, -0.1309357914639342),
  (206, -0.12731568318035072),
  (1, -0.12423464436928029),
  (178, -0.07478245937713171)]}

In [22]:
j = 0
grouped_attrs = []
for i in tqdm(range(2)):
    grouped_attrs.append([v for k, v in lime_expln.explainer_output[j].local_exp[i]])
grouped_attrs_aggr = torch.tensor([sum(ga) for ga in grouped_attrs]).to(device)
aggr_pred = torch.argmax(grouped_attrs_aggr)

  0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
aggr_pred

tensor(1, device='cuda:0')

## SHAP

In [34]:
from exlib.explainers.shap import ShapTextCls

shap_explainer = ShapTextCls(original_model, processor).to(device)



In [35]:
inputs_dict = {
    'input_ids': inputs,
    'token_type_ids': token_type_ids,
    'attention_mask': attention_mask
}

inputs_raw = [processor.decode(input_ids_i).replace('[CLS]', '').replace('[PAD]', '').strip() 
              for input_ids_i in inputs]

shap_expln = shap_explainer(inputs_raw, labels)

  0%|          | 0/272 [00:00<?, ?it/s]

In [36]:
shap_expln.explainer_output.values[0].shape

(17, 2)

In [37]:
# bsz = len(shap_expln.explainer_output.values)
j = 0
grouped_attrs_aggr = torch.tensor(shap_expln.explainer_output.values[j]).mean(0)
aggr_pred = torch.argmax(grouped_attrs_aggr)

In [38]:
aggr_pred

tensor(1)

In [39]:
labels

tensor([1], device='cuda:0')

In [41]:
import shap

In [40]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks):
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

# tokens = processor.convert_ids_to_tokens(inputs[0])
tokens = shap_expln.explainer_output.data[0]
printmd(show_attrs(tokens, shap_expln.attributions[0]))

<span style='background-color:rgb(255.00001899898052,-1.8998980522155762e-05,-1.8998980522155762e-05)'> </span><span style='background-color:rgb(327.6527286018245,-72.65272860182449,-72.65272860182449)'> it </span><span style='background-color:rgb(265.7481879845727,-10.748187984572724,-10.748187984572724)'> s </span><span style='background-color:rgb(177.70922014606185,77.29077985393815,77.29077985393815)'> a </span><span style='background-color:rgb(28.020851612091064,226.97914838790894,226.97914838790894)'> lovely </span><span style='background-color:rgb(106.52848072350025,148.47151927649975,148.47151927649975)'> film </span><span style='background-color:rgb(221.14634547382593,33.85365452617407,33.85365452617407)'> with </span><span style='background-color:rgb(50.45535508543253,204.54464491456747,204.54464491456747)'> lovely </span><span style='background-color:rgb(142.81062935013324,112.18937064986676,112.18937064986676)'> performances </span><span style='background-color:rgb(342.9165296209976,-87.91652962099761,-87.91652962099761)'> by </span><span style='background-color:rgb(368.9148986712098,-113.91489867120981,-113.91489867120981)'> buy </span><span style='background-color:rgb(224.96657498180866,30.033425018191338,30.033425018191338)'> and </span><span style='background-color:rgb(157.95395177789032,97.04604822210968,97.04604822210968)'> acc</span><span style='background-color:rgb(196.26526015810668,58.734739841893315,58.734739841893315)'> ors</span><span style='background-color:rgb(294.9926260113716,-39.99262601137161,-39.99262601137161)'> i </span><span style='background-color:rgb(255.00006079673767,-6.079673767089844e-05,-6.079673767089844e-05)'> [SEP]</span><span style='background-color:rgb(255.00006079673767,-6.079673767089844e-05,-6.079673767089844e-05)'> </span>

In [42]:
shap.plots.text(shap_expln.explainer_output)

In [25]:
shap_expln.explainer_output.data[0]

array(['', 'allan ', 'sat ', 'down ', 'at ', 'his ', 'desk ', 'and ',
       'pulled ', 'the ', 'chair ', 'in ', 'close', '. ', 'opening ',
       'a ', 'side ', 'drawer', ', ', 'he ', 'took ', 'out ', 'a ',
       'piece ', 'of ', 'paper ', 'and ', 'his ', 'ink', 'pot', '. ',
       'after ', 'filling ', 'his ', 'pen', ', ', 'allan ', 'looked ',
       'at ', 'his ', 'paper ', 'in ', 'the ', 'orange ', 'glow ',
       'from ', 'the ', 'lantern ', 'set ', 'back ', 'in ', 'the ',
       'desk', "'", 's ', 'right ', '- ', 'hand ', 'corner', '. ', 'his ',
       'pen ', 'cast ', 'a ', 'forbid', 'ding ', 'line ', 'of ',
       'shadow ', 'slant', 'ing ', 'across ', 'the ', 'page', ', ',
       'echoing ', 'the ', 'ink', 'y ', 'darkness ', 'crouch', 'ing ',
       'in ', 'the ', 'edges ', 'of ', 'the ', 'lantern', "'", 's ',
       'struggling ', 'glow', '. ', 'the ', 'only ', 'other ',
       'illumination ', 'came ', 'from ', 'a ', 'lu', 'rid ',
       'moonlight ', 'filtered ', 'through 

In [128]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks):
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

tokens = processor.convert_ids_to_tokens(inputs[0])
# tokens = shap_expln.explainer_output.data[0]
printmd(show_attrs(tokens, shap_expln.attributions[0]))

<span style='background-color:rgb(256.24154931286944,-1.2415493128694244,-1.2415493128694244)'> [CLS]</span><span style='background-color:rgb(256.24154931286944,-1.2415493128694244,-1.2415493128694244)'> allan</span><span style='background-color:rgb(256.24154931286944,-1.2415493128694244,-1.2415493128694244)'> sat</span><span style='background-color:rgb(256.24154931286944,-1.2415493128694244,-1.2415493128694244)'> down</span><span style='background-color:rgb(256.24154931286944,-1.2415493128694242,-1.2415493128694242)'> at</span><span style='background-color:rgb(256.24154931286944,-1.2415493128694242,-1.2415493128694242)'> his</span><span style='background-color:rgb(256.24154931286944,-1.2415493128694242,-1.2415493128694242)'> desk</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> and</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> pulled</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> the</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> chair</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> in</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> close</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> .</span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> opening</span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> a</span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> side</span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> drawer</span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> ,</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> he</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> took</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> out</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> a</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> piece</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> of</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> paper</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> and</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> his</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> ink</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> ##pot</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> .</span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> after</span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> filling</span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> his</span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> pen</span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> ,</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> allan</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> looked</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> at</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> his</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> paper</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> in</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> the</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> orange</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255216,-0.9614420980255216)'> glow</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255216,-0.9614420980255216)'> from</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255216,-0.9614420980255216)'> the</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255216,-0.9614420980255216)'> lantern</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> set</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> back</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> in</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> the</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> desk</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> '</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> s</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> right</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> -</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> hand</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> corner</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> .</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> his</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> pen</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> cast</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> a</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> forbid</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> ##ding</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> line</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> of</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> shadow</span><span style='background-color:rgb(254.9495958155021,0.050404184497892864,0.050404184497892864)'> slant</span><span style='background-color:rgb(254.9495958155021,0.050404184497892864,0.050404184497892864)'> ##ing</span><span style='background-color:rgb(254.9495958155021,0.05040418449789286,0.05040418449789286)'> across</span><span style='background-color:rgb(254.9495958155021,0.05040418449789286,0.05040418449789286)'> the</span><span style='background-color:rgb(254.9495958155021,0.05040418449789286,0.05040418449789286)'> page</span><span style='background-color:rgb(254.9495958155021,0.050404184497892864,0.050404184497892864)'> ,</span><span style='background-color:rgb(255.4279830811235,-0.42798308112348127,-0.42798308112348127)'> echoing</span><span style='background-color:rgb(255.4279830811235,-0.42798308112348127,-0.42798308112348127)'> the</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> ink</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> ##y</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> darkness</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> crouch</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> ##ing</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> in</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> the</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> edges</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> of</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> the</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> lantern</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> '</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> s</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> struggling</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> glow</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> .</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> the</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> only</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> other</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> illumination</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> came</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> from</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> a</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> lu</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> ##rid</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> moonlight</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> filtered</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> through</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> thin</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> branches</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> and</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> clouds</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> ,</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> casting</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> its</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> bone</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> -</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> pale</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> glow</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> onto</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> the</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> pine</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> floor</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> ##boards</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> .</span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> allan</span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> unfolded</span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> another</span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> page</span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> ,</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> this</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> one</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> crowded</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> with</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> ranks</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> of</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> letters</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> in</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> tight</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> formation</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> from</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> left</span><span style='background-color:rgb(254.98819142421326,0.011808575786726198,0.011808575786726198)'> to</span><span style='background-color:rgb(254.98819142421326,0.011808575786726198,0.011808575786726198)'> right</span><span style='background-color:rgb(254.98819142421326,0.011808575786726198,0.011808575786726198)'> .</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> the</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> lines</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> of</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> letters</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> stepped</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> into</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> their</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> divisions</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> ,</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> in</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> the</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> shape</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> of</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> a</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> story</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> '</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> s</span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> outline</span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> :</span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> the</span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> loose</span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> ,</span><span style='background-color:rgb(253.83068933855296,1.169310661447053,1.169310661447053)'> dry</span><span style='background-color:rgb(253.83068933855296,1.169310661447053,1.169310661447053)'> skeleton</span><span style='background-color:rgb(253.83068933855296,1.169310661447053,1.169310661447053)'> of</span><span style='background-color:rgb(253.83068933855296,1.169310661447053,1.169310661447053)'> a</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> tale</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> lay</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> exposed</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> beneath</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> their</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> feet</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> ,</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> awaiting</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> tend</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> ##ons</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> ,</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> muscle</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> and</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> blushing</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> skin</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> .</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> allan</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> reviewed</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> the</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> troops</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> ,</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> all</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> prepared</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> to</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> di</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> ##se</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> ##mba</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> ##rk</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> ,</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> their</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> task</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> to</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> form</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> the</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> tale</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> of</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> a</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> young</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> man</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> returning</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> home</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> from</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> life</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> abroad</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> to</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> find</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> his</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> childhood</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> friend</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308822,-0.5538878255308822)'> a</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308822,-0.5538878255308822)'> bride</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> to</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> -</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> be</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> ,</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> thus</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> upset</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> ##ting</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818707,-0.11904207885818707)'> the</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818707,-0.11904207885818707)'> apple</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> cart</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> of</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> his</span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> life</span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> '</span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> s</span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> plan</span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> ,</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> clarified</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> –</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> of</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> course</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> –</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> by</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> his</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> very</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> time</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> away</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> from</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> her</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> he</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> loved</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> best</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> .</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> although</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> the</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> concept</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> was</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> a</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> simple</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> one</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> ,</span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> allan</span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> thought</span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> it</span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> had</span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> potential</span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> .</span><span style='background-color:rgb(255.37261852712808,-0.3726185271280588,-0.3726185271280588)'> besides</span><span style='background-color:rgb(255.37261852712808,-0.3726185271280588,-0.3726185271280588)'> ,</span><span style='background-color:rgb(255.37261852712808,-0.3726185271280588,-0.3726185271280588)'> the</span><span style='background-color:rgb(255.37261852712808,-0.3726185271280588,-0.3726185271280588)'> public</span><span style='background-color:rgb(255.37261852712808,-0.37261852712805876,-0.37261852712805876)'> liked</span><span style='background-color:rgb(255.37261852712808,-0.37261852712805876,-0.37261852712805876)'> a</span><span style='background-color:rgb(255.37261852712808,-0.37261852712805876,-0.37261852712805876)'> good</span><span style='background-color:rgb(255.37261852712808,-0.37261852712805876,-0.37261852712805876)'> ,</span><span style='background-color:rgb(255.1774729725203,-0.17747297252032468,-0.17747297252032468)'> simple</span><span style='background-color:rgb(255.1774729725203,-0.17747297252032468,-0.17747297252032468)'> romance</span><span style='background-color:rgb(255.1774729725203,-0.17747297252032468,-0.17747297252032468)'> .</span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> perhaps</span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> this</span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> will</span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> be</span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> more</span><span style='background-color:rgb(255.67398949842237,-0.6739894984223479,-0.6739894984223479)'> sale</span><span style='background-color:rgb(255.67398949842237,-0.6739894984223479,-0.6739894984223479)'> ##able</span><span style='background-color:rgb(255.67398949842237,-0.6739894984223479,-0.6739894984223479)'> ,</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> he</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> thought</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> and</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> began</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> to</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> write</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> .</span><span style='background-color:rgb(253.2792412531055,1.7207587468944756,1.7207587468944756)'> they</span><span style='background-color:rgb(253.2792412531055,1.7207587468944756,1.7207587468944756)'> gazed</span><span style='background-color:rgb(253.2792412531055,1.7207587468944754,1.7207587468944754)'> at</span><span style='background-color:rgb(253.2792412531055,1.7207587468944754,1.7207587468944754)'> each</span><span style='background-color:rgb(253.2792412531055,1.7207587468944754,1.7207587468944754)'> other</span><span style='background-color:rgb(253.2792412531055,1.7207587468944754,1.7207587468944754)'> ,</span><span style='background-color:rgb(253.52325625965358,1.4767437403464156,1.4767437403464156)'> lost</span><span style='background-color:rgb(253.52325625965358,1.4767437403464156,1.4767437403464156)'> in</span><span style='background-color:rgb(253.52325625965358,1.4767437403464156,1.4767437403464156)'> the</span><span style='background-color:rgb(253.52325625965358,1.4767437403464154,1.4767437403464154)'> rap</span><span style='background-color:rgb(253.52325625965358,1.4767437403464154,1.4767437403464154)'> ##ture</span><span style='background-color:rgb(253.52325625965358,1.4767437403464154,1.4767437403464154)'> of</span><span style='background-color:rgb(253.52325625965358,1.4767437403464154,1.4767437403464154)'> love</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> based</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> so</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> deeply</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> within</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> their</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> hearts</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> that</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> they</span><span style='background-color:rgb(253.86358484507656,1.1364151549234183,1.1364151549234183)'> had</span><span style='background-color:rgb(253.86358484507656,1.1364151549234183,1.1364151549234183)'> never</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> seen</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> it</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> before</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> .</span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> "</span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> what</span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> about</span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> roger</span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> ?</span><span style='background-color:rgb(247.2031556009855,7.796844399014503,7.796844399014503)'> "</span><span style='background-color:rgb(247.2031556009855,7.796844399014503,7.796844399014503)'> she</span><span style='background-color:rgb(247.2031556009855,7.796844399014503,7.796844399014503)'> asked</span><span style='background-color:rgb(247.2031556009855,7.796844399014502,7.796844399014502)'> ,</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> knowing</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> that</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> the</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> answer</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> no</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> longer</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> mattered</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> .</span><span style='background-color:rgb(180.79991474747658,74.20008525252342,74.20008525252342)'> [SEP]</span><span style='background-color:rgb(252.53304625558357,2.466953744416449,2.466953744416449)'> name</span><span style='background-color:rgb(252.53304625558357,2.466953744416449,2.466953744416449)'> few</span><span style='background-color:rgb(257.6414919008935,-2.6414919008935356,-2.6414919008935356)'> objects</span><span style='background-color:rgb(257.6414919008935,-2.6414919008935356,-2.6414919008935356)'> said</span><span style='background-color:rgb(255.21320026141308,-0.21320026141308648,-0.21320026141308648)'> to</span><span style='background-color:rgb(255.21320026141308,-0.21320026141308648,-0.21320026141308648)'> be</span><span style='background-color:rgb(255.21320026141308,-0.21320026141308648,-0.21320026141308648)'> in</span><span style='background-color:rgb(261.44247199830716,-6.4424719983071554,-6.4424719983071554)'> or</span><span style='background-color:rgb(259.5920876642049,-4.5920876642048825,-4.5920876642048825)'> on</span><span style='background-color:rgb(255.37087327291374,-0.37087327291374095,-0.37087327291374095)'> allan</span><span style='background-color:rgb(255.37087327291374,-0.37087327291374095,-0.37087327291374095)'> '</span><span style='background-color:rgb(255.37087327291374,-0.37087327291374095,-0.37087327291374095)'> s</span><span style='background-color:rgb(250.95823206673958,4.041767933260417,4.041767933260417)'> desk</span><span style='background-color:rgb(250.95823206673958,4.041767933260417,4.041767933260417)'> |</span><span style='background-color:rgb(253.25270082769566,1.7472991723043376,1.7472991723043376)'> |</span><span style='background-color:rgb(225.7652162591403,29.23478374085971,29.23478374085971)'> erase</span><span style='background-color:rgb(225.7652162591403,29.23478374085971,29.23478374085971)'> ##r</span><span style='background-color:rgb(255.000018524006,-1.8524006009101868e-05,-1.8524006009101868e-05)'> [SEP]</span><span style='background-color:rgb(255.000018524006,-1.8524006009101868e-05,-1.8524006009101868e-05)'> [PAD]</span> [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

## RISE

In [43]:
class WrappedBackboneModel2(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, inputs=None, **kwargs):
        outputs = self.model(inputs, output_hidden_states=True, **kwargs)
        # import pdb; pdb.set_trace()
        return torch.softmax(outputs.logits, dim=-1)

In [44]:
original_model2 = WrappedBackboneModel2(backbone_model)

In [54]:
from exlib.explainers.common import FeatureAttrMethod, FeatureAttrOutput
from skimage.transform import resize

class RiseTextCls(FeatureAttrMethod):
    def __init__(self, model, input_size=512, \
                 gpu_batch=100, N=500, \
                 s=8, p1=0.5, seed=42, mask_combine=None,
                normalize=False):
        super(RiseTextCls, self).__init__(model)
        self.input_size = input_size  # only one number for text
        self.gpu_batch = gpu_batch
        self.generate_masks(N, s, p1)
        self.mask_combine = mask_combine
        self.normalize = normalize

    def generate_masks(self, N, s, p1, savepath='masks.npy'):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        cell_size = np.ceil(np.array(self.input_size) / s)
        up_size = (s + 1) * cell_size

        grid = np.random.rand(N, s) < p1
        grid = grid.astype('float32')

        self.masks = np.empty((N, self.input_size))

        for i in range(N):
            # Random shifts
            x = np.random.randint(0, cell_size)
            # Linear upsampling and cropping
            self.masks[i, :] = resize(grid[i], tuple([up_size]), order=1, mode='reflect', 
                                        anti_aliasing=False)[x:x + self.input_size]
        self.masks = (self.masks.reshape(-1, self.input_size) >= 0.5)
        # np.save(savepath, self.masks)
        self.masks = torch.from_numpy(self.masks).long()
        self.masks = self.masks.to(device)
        self.N = N
        self.p1 = p1

    def forward(self, x, label=None, kwargs=None):
        # Apply array of filters to the image]
        # print('RISE')
        self.model.eval()
        with torch.no_grad():
            N = self.N
            B, L = x.size()
            if self.mask_combine:
                stack = self.mask_combine(x, self.masks)
                stack = stack.reshape(N * B, L, -1)
            else:
                stack = torch.mul(self.masks.view(N, 1, L), x.data.view(B, L))
                # import pdb; pdb.set_trace()
                stack = stack.reshape(N * B, L)
            kwargs_new = {}
            for k, v in kwargs.items():
                if v is not None:
                    kwargs_new[k] = v.unsqueeze(0).expand(N, B, L).reshape(N * B, L)
            # stack = stack

            #p = nn.Softmax(dim=1)(model(stack)) in batches
            p = []
            for i in tqdm(range(0, N*B, self.gpu_batch)):
                kwargs_curr = {k: v[i:min(i + self.gpu_batch, N*B)] for k, v in kwargs_new.items()}
                if self.mask_combine:
                    pred = self.model(inputs_embeds=stack[i:min(i + self.gpu_batch, N*B)], **kwargs_curr)
                else:
                    pred = self.model(stack[i:min(i + self.gpu_batch, N*B)], **kwargs_curr)
                p.append(pred)
            p = torch.cat(p)
            if label is None:
                # if no label, then explain the top class
                pred_x = self.model(x, **kwargs)
                label = torch.argmax(pred_x, dim=-1)
            CL = p.size(1)
            p = p.view(N, B, CL)
            sal = torch.matmul(p.permute(1, 2, 0), self.masks.float().view(N, L))
            # import pdb; pdb.set_trace()
            if self.normalize:
                sal = sal / L
            sal = sal.view(B, CL, L)
        
        return FeatureAttrOutput(sal[range(B), label], sal)


In [51]:
from exlib.explainers.rise import RiseTextCls

In [55]:
inputs.shape, labels.shape, kwargs

(torch.Size([1, 16]),
 torch.Size([1]),
 {'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')})

In [56]:
# from exlib.explainers.rise import RiseTextCls

rise_explainer = RiseTextCls(original_model2).to(device)
rise_expln = rise_explainer(inputs, labels, kwargs=kwargs)

RuntimeError: shape '[500, 1, 16]' is invalid for input of size 256000

In [86]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks, normalize=True):
    if normalize:
        masks_max = masks.max(-1).values
        masks_min = masks.min(-1).values
        # import pdb; pdb.set_trace()
        masks = (masks - masks_min) / (masks_max - masks_min)
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

tokens = processor.convert_ids_to_tokens(inputs[0])
# tokens = shap_expln.explainer_output.data[0]
printmd(show_attrs(tokens, rise_expln.attributions[0]))

<span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> [CLS]</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> allan</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> sat</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> down</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> at</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> his</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> desk</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> and</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> pulled</span><span style='background-color:rgb(239.05032735317945,15.949672646820545,15.949672646820545)'> the</span><span style='background-color:rgb(238.56281351298094,16.437186487019062,16.437186487019062)'> chair</span><span style='background-color:rgb(240.37113362923265,14.628866370767355,14.628866370767355)'> in</span><span style='background-color:rgb(240.08213253691792,14.917867463082075,14.917867463082075)'> close</span><span style='background-color:rgb(240.31039768829942,14.689602311700583,14.689602311700583)'> .</span><span style='background-color:rgb(241.54580734670162,13.454192653298378,13.454192653298378)'> opening</span><span style='background-color:rgb(237.18802828341722,17.811971716582775,17.811971716582775)'> a</span><span style='background-color:rgb(236.88218649476767,18.117813505232334,18.117813505232334)'> side</span><span style='background-color:rgb(239.54216916114092,15.457830838859081,15.457830838859081)'> drawer</span><span style='background-color:rgb(234.3964909762144,20.60350902378559,20.60350902378559)'> ,</span><span style='background-color:rgb(234.2841386049986,20.71586139500141,20.71586139500141)'> he</span><span style='background-color:rgb(240.9393845871091,14.060615412890911,14.060615412890911)'> took</span><span style='background-color:rgb(234.96054980903864,20.03945019096136,20.03945019096136)'> out</span><span style='background-color:rgb(232.37581450492144,22.624185495078564,22.624185495078564)'> a</span><span style='background-color:rgb(232.15826477855444,22.84173522144556,22.84173522144556)'> piece</span><span style='background-color:rgb(231.23348750174046,23.766512498259544,23.766512498259544)'> of</span><span style='background-color:rgb(230.2177906036377,24.782209396362305,24.782209396362305)'> paper</span><span style='background-color:rgb(228.52032478898764,26.479675211012363,26.479675211012363)'> and</span><span style='background-color:rgb(228.07790502905846,26.922094970941544,26.922094970941544)'> his</span><span style='background-color:rgb(223.72059144079685,31.279408559203148,31.279408559203148)'> ink</span><span style='background-color:rgb(221.895929351449,33.10407064855099,33.10407064855099)'> ##pot</span><span style='background-color:rgb(222.2952537238598,32.70474627614021,32.70474627614021)'> .</span><span style='background-color:rgb(221.4844912290573,33.51550877094269,33.51550877094269)'> after</span><span style='background-color:rgb(225.60621935874224,29.393780641257763,29.393780641257763)'> filling</span><span style='background-color:rgb(225.0268853455782,29.973114654421806,29.973114654421806)'> his</span><span style='background-color:rgb(225.81458307802677,29.18541692197323,29.18541692197323)'> pen</span><span style='background-color:rgb(227.2758972644806,27.72410273551941,27.72410273551941)'> ,</span><span style='background-color:rgb(227.29177091270685,27.708229087293148,27.708229087293148)'> allan</span><span style='background-color:rgb(227.47620731592178,27.523792684078217,27.523792684078217)'> looked</span><span style='background-color:rgb(218.09501990675926,36.90498009324074,36.90498009324074)'> at</span><span style='background-color:rgb(220.64218282699585,34.35781717300415,34.35781717300415)'> his</span><span style='background-color:rgb(222.8205071389675,32.179492861032486,32.179492861032486)'> paper</span><span style='background-color:rgb(214.9123790860176,40.08762091398239,40.08762091398239)'> in</span><span style='background-color:rgb(214.09443117678165,40.905568823218346,40.905568823218346)'> the</span><span style='background-color:rgb(213.3463691174984,41.6536308825016,41.6536308825016)'> orange</span><span style='background-color:rgb(210.39875328540802,44.60124671459198,44.60124671459198)'> glow</span><span style='background-color:rgb(210.4980267584324,44.50197324156761,44.50197324156761)'> from</span><span style='background-color:rgb(214.15965847671032,40.84034152328968,40.84034152328968)'> the</span><span style='background-color:rgb(216.51093810796738,38.48906189203262,38.48906189203262)'> lantern</span><span style='background-color:rgb(213.77601206302643,41.22398793697357,41.22398793697357)'> set</span><span style='background-color:rgb(215.50702057778835,39.49297942221165,39.49297942221165)'> back</span><span style='background-color:rgb(211.51272431015968,43.48727568984032,43.48727568984032)'> in</span><span style='background-color:rgb(205.65688893198967,49.34311106801033,49.34311106801033)'> the</span><span style='background-color:rgb(208.31500969827175,46.68499030172825,46.68499030172825)'> desk</span><span style='background-color:rgb(204.97735239565372,50.022647604346275,50.022647604346275)'> '</span><span style='background-color:rgb(202.79842771589756,52.20157228410244,52.20157228410244)'> s</span><span style='background-color:rgb(206.73006154596806,48.269938454031944,48.269938454031944)'> right</span><span style='background-color:rgb(209.32514749467373,45.67485250532627,45.67485250532627)'> -</span><span style='background-color:rgb(209.2138286679983,45.786171332001686,45.786171332001686)'> hand</span><span style='background-color:rgb(211.4594093710184,43.54059062898159,43.54059062898159)'> corner</span><span style='background-color:rgb(216.85039289295673,38.149607107043266,38.149607107043266)'> .</span><span style='background-color:rgb(216.5876141935587,38.41238580644131,38.41238580644131)'> his</span><span style='background-color:rgb(218.05438488721848,36.945615112781525,36.945615112781525)'> pen</span><span style='background-color:rgb(223.2751413434744,31.724858656525612,31.724858656525612)'> cast</span><span style='background-color:rgb(225.45326616615057,29.54673383384943,29.54673383384943)'> a</span><span style='background-color:rgb(218.73076379299164,36.26923620700836,36.26923620700836)'> forbid</span><span style='background-color:rgb(219.4447796791792,35.55522032082081,35.55522032082081)'> ##ding</span><span style='background-color:rgb(222.72878766059875,32.271212339401245,32.271212339401245)'> line</span><span style='background-color:rgb(217.83161044120789,37.168389558792114,37.168389558792114)'> of</span><span style='background-color:rgb(215.65235517919064,39.347644820809364,39.347644820809364)'> shadow</span><span style='background-color:rgb(219.77960631251335,35.22039368748665,35.22039368748665)'> slant</span><span style='background-color:rgb(226.28446016460657,28.71553983539343,28.71553983539343)'> ##ing</span><span style='background-color:rgb(226.13490018993616,28.86509981006384,28.86509981006384)'> across</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> the</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> page</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> ,</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> echoing</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> the</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> ink</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> ##y</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> darkness</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> crouch</span><span style='background-color:rgb(227.92707972228527,27.07292027771473,27.07292027771473)'> ##ing</span><span style='background-color:rgb(225.29844917356968,29.70155082643032,29.70155082643032)'> in</span><span style='background-color:rgb(224.4621292501688,30.5378707498312,30.5378707498312)'> the</span><span style='background-color:rgb(224.46542367339134,30.534576326608658,30.534576326608658)'> edges</span><span style='background-color:rgb(224.0126532688737,30.98734673112631,30.98734673112631)'> of</span><span style='background-color:rgb(224.13079652935266,30.869203470647335,30.869203470647335)'> the</span><span style='background-color:rgb(223.90056688338518,31.09943311661482,31.09943311661482)'> lantern</span><span style='background-color:rgb(227.8220495581627,27.17795044183731,27.17795044183731)'> '</span><span style='background-color:rgb(226.9133796170354,28.08662038296461,28.08662038296461)'> s</span><span style='background-color:rgb(233.22248317301273,21.777516826987267,21.777516826987267)'> struggling</span><span style='background-color:rgb(232.6533668115735,22.346633188426495,22.346633188426495)'> glow</span><span style='background-color:rgb(227.92012609541416,27.07987390458584,27.07987390458584)'> .</span><span style='background-color:rgb(225.74080243706703,29.259197562932968,29.259197562932968)'> the</span><span style='background-color:rgb(230.94674918800592,24.053250811994076,24.053250811994076)'> only</span><span style='background-color:rgb(230.94674918800592,24.053250811994076,24.053250811994076)'> other</span><span style='background-color:rgb(230.96072673797607,24.039273262023926,24.039273262023926)'> illumination</span><span style='background-color:rgb(234.6109799668193,20.389020033180714,20.389020033180714)'> came</span><span style='background-color:rgb(232.34769221395254,22.65230778604746,22.65230778604746)'> from</span><span style='background-color:rgb(234.91182882338762,20.088171176612377,20.088171176612377)'> a</span><span style='background-color:rgb(238.9669617265463,16.033038273453712,16.033038273453712)'> lu</span><span style='background-color:rgb(236.29400085657835,18.70599914342165,18.70599914342165)'> ##rid</span><span style='background-color:rgb(237.949135042727,17.050864957273006,17.050864957273006)'> moonlight</span><span style='background-color:rgb(234.4561629742384,20.543837025761604,20.543837025761604)'> filtered</span><span style='background-color:rgb(229.95048824697733,25.04951175302267,25.04951175302267)'> through</span><span style='background-color:rgb(228.4090382605791,26.59096173942089,26.59096173942089)'> thin</span><span style='background-color:rgb(227.36931335180998,27.63068664819002,27.63068664819002)'> branches</span><span style='background-color:rgb(229.9918870255351,25.008112974464893,25.008112974464893)'> and</span><span style='background-color:rgb(227.79579296708107,27.20420703291893,27.20420703291893)'> clouds</span><span style='background-color:rgb(227.53883935511112,27.461160644888878,27.461160644888878)'> ,</span><span style='background-color:rgb(231.5320735797286,23.467926420271397,23.467926420271397)'> casting</span><span style='background-color:rgb(228.9858416095376,26.0141583904624,26.0141583904624)'> its</span><span style='background-color:rgb(228.7048276886344,26.295172311365604,26.295172311365604)'> bone</span><span style='background-color:rgb(231.5696782618761,23.430321738123894,23.430321738123894)'> -</span><span style='background-color:rgb(235.56191504001617,19.438084959983826,19.438084959983826)'> pale</span><span style='background-color:rgb(236.0330555588007,18.966944441199303,18.966944441199303)'> glow</span><span style='background-color:rgb(236.26880820840597,18.73119179159403,18.73119179159403)'> onto</span><span style='background-color:rgb(238.34815923124552,16.651840768754482,16.651840768754482)'> the</span><span style='background-color:rgb(238.3444981276989,16.6555018723011,16.6555018723011)'> pine</span><span style='background-color:rgb(233.9514759555459,21.048524044454098,21.048524044454098)'> floor</span><span style='background-color:rgb(236.1279022693634,18.872097730636597,18.872097730636597)'> ##boards</span><span style='background-color:rgb(232.21790067851543,22.782099321484566,22.782099321484566)'> .</span><span style='background-color:rgb(234.161570481956,20.838429518043995,20.838429518043995)'> allan</span><span style='background-color:rgb(236.34332221001387,18.656677789986134,18.656677789986134)'> unfolded</span><span style='background-color:rgb(234.16596304625273,20.834036953747272,20.834036953747272)'> another</span><span style='background-color:rgb(239.97580379247665,15.024196207523346,15.024196207523346)'> page</span><span style='background-color:rgb(244.29617671296,10.703823287039995,10.703823287039995)'> ,</span><span style='background-color:rgb(239.85526476055384,15.144735239446163,15.144735239446163)'> this</span><span style='background-color:rgb(241.61479644477367,13.385203555226326,13.385203555226326)'> one</span><span style='background-color:rgb(238.18701937794685,16.812980622053146,16.812980622053146)'> crowded</span><span style='background-color:rgb(238.08544892817736,16.914551071822643,16.914551071822643)'> with</span><span style='background-color:rgb(238.4376159310341,16.562384068965912,16.562384068965912)'> ranks</span><span style='background-color:rgb(238.74442096799612,16.25557903200388,16.25557903200388)'> of</span><span style='background-color:rgb(241.26878701150417,13.731212988495827,13.731212988495827)'> letters</span><span style='background-color:rgb(240.57167736813426,14.42832263186574,14.42832263186574)'> in</span><span style='background-color:rgb(238.39461743831635,16.605382561683655,16.605382561683655)'> tight</span><span style='background-color:rgb(240.45336786657572,14.546632133424282,14.546632133424282)'> formation</span><span style='background-color:rgb(239.8110693320632,15.188930667936802,15.188930667936802)'> from</span><span style='background-color:rgb(239.8110693320632,15.188930667936802,15.188930667936802)'> left</span><span style='background-color:rgb(238.25298003852367,16.747019961476326,16.747019961476326)'> to</span><span style='background-color:rgb(240.51070868968964,14.489291310310364,14.489291310310364)'> right</span><span style='background-color:rgb(229.49668619781733,25.503313802182674,25.503313802182674)'> .</span><span style='background-color:rgb(225.00219237059355,29.997807629406452,29.997807629406452)'> the</span><span style='background-color:rgb(224.91326954215765,30.08673045784235,30.08673045784235)'> lines</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> of</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> letters</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> stepped</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> into</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> their</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> divisions</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> ,</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> in</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> the</span><span style='background-color:rgb(231.04232925921679,23.957670740783215,23.957670740783215)'> shape</span><span style='background-color:rgb(233.70880007743835,21.291199922561646,21.291199922561646)'> of</span><span style='background-color:rgb(232.0658803358674,22.934119664132595,22.934119664132595)'> a</span><span style='background-color:rgb(234.2390464246273,20.760953575372696,20.760953575372696)'> story</span><span style='background-color:rgb(234.69181682914495,20.308183170855045,20.308183170855045)'> '</span><span style='background-color:rgb(234.69181682914495,20.308183170855045,20.308183170855045)'> s</span><span style='background-color:rgb(237.22936056554317,17.770639434456825,17.770639434456825)'> outline</span><span style='background-color:rgb(233.22308164089918,21.77691835910082,21.77691835910082)'> :</span><span style='background-color:rgb(231.1536157876253,23.846384212374687,23.846384212374687)'> the</span><span style='background-color:rgb(225.6213292479515,29.378670752048492,29.378670752048492)'> loose</span><span style='background-color:rgb(225.96204556524754,29.037954434752464,29.037954434752464)'> ,</span><span style='background-color:rgb(230.23672688752413,24.763273112475872,24.763273112475872)'> dry</span><span style='background-color:rgb(234.02545608580112,20.974543914198875,20.974543914198875)'> skeleton</span><span style='background-color:rgb(233.1605825945735,21.839417405426502,21.839417405426502)'> of</span><span style='background-color:rgb(232.87897020578384,22.121029794216156,22.121029794216156)'> a</span><span style='background-color:rgb(234.12010330706835,20.879896692931652,20.879896692931652)'> tale</span><span style='background-color:rgb(232.64661077409983,22.353389225900173,22.353389225900173)'> lay</span><span style='background-color:rgb(239.1263717226684,15.87362827733159,15.87362827733159)'> exposed</span><span style='background-color:rgb(239.4784712791443,15.521528720855713,15.521528720855713)'> beneath</span><span style='background-color:rgb(239.76933617144823,15.23066382855177,15.23066382855177)'> their</span><span style='background-color:rgb(243.95129961892962,11.048700381070375,11.048700381070375)'> feet</span><span style='background-color:rgb(236.70716978609562,18.29283021390438,18.29283021390438)'> ,</span><span style='background-color:rgb(238.7083115056157,16.29168849438429,16.29168849438429)'> awaiting</span><span style='background-color:rgb(236.7869085073471,18.213091492652893,18.213091492652893)'> tend</span><span style='background-color:rgb(238.49252678453922,16.507473215460777,16.507473215460777)'> ##ons</span><span style='background-color:rgb(240.11631075292826,14.883689247071743,14.883689247071743)'> ,</span><span style='background-color:rgb(240.37975251674652,14.620247483253479,14.620247483253479)'> muscle</span><span style='background-color:rgb(240.37902105599642,14.620978944003582,14.620978944003582)'> and</span><span style='background-color:rgb(240.64419362694025,14.35580637305975,14.35580637305975)'> blushing</span><span style='background-color:rgb(242.61827984824777,12.381720151752234,12.381720151752234)'> skin</span><span style='background-color:rgb(243.03088165819645,11.96911834180355,11.96911834180355)'> .</span><span style='background-color:rgb(240.48222161829472,14.517778381705284,14.517778381705284)'> allan</span><span style='background-color:rgb(244.09796319901943,10.902036800980568,10.902036800980568)'> reviewed</span><span style='background-color:rgb(242.34565302729607,12.654346972703934,12.654346972703934)'> the</span><span style='background-color:rgb(240.32018216326833,14.679817836731672,14.679817836731672)'> troops</span><span style='background-color:rgb(240.32018216326833,14.679817836731672,14.679817836731672)'> ,</span><span style='background-color:rgb(238.2408320903778,16.759167909622192,16.759167909622192)'> all</span><span style='background-color:rgb(233.88608146458864,21.113918535411358,21.113918535411358)'> prepared</span><span style='background-color:rgb(236.23646054416895,18.76353945583105,18.76353945583105)'> to</span><span style='background-color:rgb(236.0796144604683,18.920385539531708,18.920385539531708)'> di</span><span style='background-color:rgb(231.6441599652171,23.355840034782887,23.355840034782887)'> ##se</span><span style='background-color:rgb(229.3445309624076,25.65546903759241,25.65546903759241)'> ##mba</span><span style='background-color:rgb(235.86698926985264,19.13301073014736,19.13301073014736)'> ##rk</span><span style='background-color:rgb(238.04378226399422,16.956217736005783,16.956217736005783)'> ,</span><span style='background-color:rgb(230.11156160384417,24.888438396155834,24.888438396155834)'> their</span><span style='background-color:rgb(232.3241961747408,22.67580382525921,22.67580382525921)'> task</span><span style='background-color:rgb(234.68685809522867,20.313141904771328,20.313141904771328)'> to</span><span style='background-color:rgb(232.35378328710794,22.646216712892056,22.646216712892056)'> form</span><span style='background-color:rgb(236.9253844767809,18.07461552321911,18.07461552321911)'> the</span><span style='background-color:rgb(236.6703288629651,18.329671137034893,18.329671137034893)'> tale</span><span style='background-color:rgb(234.83511853963137,20.164881460368633,20.164881460368633)'> of</span><span style='background-color:rgb(236.87722966074944,18.122770339250565,18.122770339250565)'> a</span><span style='background-color:rgb(235.74911199510098,19.250888004899025,19.250888004899025)'> young</span><span style='background-color:rgb(242.72557374089956,12.274426259100437,12.274426259100437)'> man</span><span style='background-color:rgb(242.7252080105245,12.274791989475489,12.274791989475489)'> returning</span><span style='background-color:rgb(236.53747659176588,18.46252340823412,18.46252340823412)'> home</span><span style='background-color:rgb(234.50664706528187,20.493352934718132,20.493352934718132)'> from</span><span style='background-color:rgb(227.97410409897566,27.02589590102434,27.02589590102434)'> life</span><span style='background-color:rgb(231.79292008280754,23.20707991719246,23.20707991719246)'> abroad</span><span style='background-color:rgb(231.82047430425882,23.179525695741177,23.179525695741177)'> to</span><span style='background-color:rgb(236.1786200478673,18.821379952132702,18.821379952132702)'> find</span><span style='background-color:rgb(240.67281369119883,14.327186308801174,14.327186308801174)'> his</span><span style='background-color:rgb(240.60099659487605,14.399003405123949,14.399003405123949)'> childhood</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> friend</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> a</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> bride</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> to</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> -</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> be</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> ,</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> thus</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> upset</span><span style='background-color:rgb(235.9937846660614,19.0062153339386,19.0062153339386)'> ##ting</span><span style='background-color:rgb(233.8148295879364,21.1851704120636,21.1851704120636)'> the</span><span style='background-color:rgb(236.28192130476236,18.718078695237637,18.718078695237637)'> apple</span><span style='background-color:rgb(238.14502023160458,16.854979768395424,16.854979768395424)'> cart</span><span style='background-color:rgb(238.0439494550228,16.956050544977188,16.956050544977188)'> of</span><span style='background-color:rgb(238.0439494550228,16.956050544977188,16.956050544977188)'> his</span><span style='background-color:rgb(234.9916321411729,20.008367858827114,20.008367858827114)'> life</span><span style='background-color:rgb(237.9959922283888,17.004007771611214,17.004007771611214)'> '</span><span style='background-color:rgb(235.8200978860259,19.179902113974094,19.179902113974094)'> s</span><span style='background-color:rgb(237.0056228712201,17.994377128779888,17.994377128779888)'> plan</span><span style='background-color:rgb(235.16738411039114,19.83261588960886,19.83261588960886)'> ,</span><span style='background-color:rgb(228.73318176716566,26.26681823283434,26.26681823283434)'> clarified</span><span style='background-color:rgb(229.110440723598,25.889559276401997,25.889559276401997)'> –</span><span style='background-color:rgb(225.1910213381052,29.8089786618948,29.8089786618948)'> of</span><span style='background-color:rgb(225.4085710644722,29.5914289355278,29.5914289355278)'> course</span><span style='background-color:rgb(222.0722132921219,32.92778670787811,32.92778670787811)'> –</span><span style='background-color:rgb(219.90919835865498,35.090801641345024,35.090801641345024)'> by</span><span style='background-color:rgb(215.94981081783772,39.050189182162285,39.050189182162285)'> his</span><span style='background-color:rgb(215.6097898632288,39.3902101367712,39.3902101367712)'> very</span><span style='background-color:rgb(215.32005541026592,39.67994458973408,39.67994458973408)'> time</span><span style='background-color:rgb(208.9627305418253,46.037269458174706,46.037269458174706)'> away</span><span style='background-color:rgb(214.87457491457462,40.12542508542538,40.12542508542538)'> from</span><span style='background-color:rgb(212.4018234014511,42.59817659854889,42.59817659854889)'> her</span><span style='background-color:rgb(216.88430227339268,38.11569772660732,38.11569772660732)'> he</span><span style='background-color:rgb(216.77870593965054,38.221294060349464,38.221294060349464)'> loved</span><span style='background-color:rgb(217.02371299266815,37.97628700733185,37.97628700733185)'> best</span><span style='background-color:rgb(216.57559923827648,38.42440076172352,38.42440076172352)'> .</span><span style='background-color:rgb(218.7565530091524,36.24344699084759,36.24344699084759)'> although</span><span style='background-color:rgb(216.5697095543146,38.43029044568539,38.43029044568539)'> the</span><span style='background-color:rgb(208.02451148629189,46.975488513708115,46.975488513708115)'> concept</span><span style='background-color:rgb(209.96498376131058,45.03501623868942,45.03501623868942)'> was</span><span style='background-color:rgb(209.96498376131058,45.03501623868942,45.03501623868942)'> a</span><span style='background-color:rgb(208.00783798098564,46.99216201901436,46.99216201901436)'> simple</span><span style='background-color:rgb(209.76131089031696,45.23868910968304,45.23868910968304)'> one</span><span style='background-color:rgb(213.43492716550827,41.56507283449173,41.56507283449173)'> ,</span><span style='background-color:rgb(215.31133867800236,39.68866132199764,39.68866132199764)'> allan</span><span style='background-color:rgb(215.21203100681305,39.78796899318695,39.78796899318695)'> thought</span><span style='background-color:rgb(211.54500357806683,43.454996421933174,43.454996421933174)'> it</span><span style='background-color:rgb(211.26339308917522,43.736606910824776,43.736606910824776)'> had</span><span style='background-color:rgb(206.8249424546957,48.1750575453043,48.1750575453043)'> potential</span><span style='background-color:rgb(215.76467335224152,39.235326647758484,39.235326647758484)'> .</span><span style='background-color:rgb(220.4708918184042,34.5291081815958,34.5291081815958)'> besides</span><span style='background-color:rgb(215.13099655508995,39.86900344491005,39.86900344491005)'> ,</span><span style='background-color:rgb(217.30832152068615,37.69167847931385,37.69167847931385)'> the</span><span style='background-color:rgb(217.3394038528204,37.660596147179604,37.660596147179604)'> public</span><span style='background-color:rgb(212.98342019319534,42.01657980680466,42.01657980680466)'> liked</span><span style='background-color:rgb(211.12937428057194,43.87062571942806,43.87062571942806)'> a</span><span style='background-color:rgb(208.69376577436924,46.30623422563076,46.30623422563076)'> good</span><span style='background-color:rgb(206.61860972642899,48.381390273571014,48.381390273571014)'> ,</span><span style='background-color:rgb(207.09414280951023,47.90585719048977,47.90585719048977)'> simple</span><span style='background-color:rgb(205.4281260073185,49.5718739926815,49.5718739926815)'> romance</span><span style='background-color:rgb(203.29489767551422,51.70510232448578,51.70510232448578)'> .</span><span style='background-color:rgb(198.00821095705032,56.99178904294968,56.99178904294968)'> perhaps</span><span style='background-color:rgb(193.52942928671837,61.47057071328163,61.47057071328163)'> this</span><span style='background-color:rgb(193.5286959260702,61.47130407392979,61.47130407392979)'> will</span><span style='background-color:rgb(201.9247367978096,53.0752632021904,53.0752632021904)'> be</span><span style='background-color:rgb(197.67501443624496,57.324985563755035,57.324985563755035)'> more</span><span style='background-color:rgb(197.67501443624496,57.324985563755035,57.324985563755035)'> sale</span><span style='background-color:rgb(193.85623455047607,61.143765449523926,61.143765449523926)'> ##able</span><span style='background-color:rgb(189.49998870491982,65.50001129508018,65.50001129508018)'> ,</span><span style='background-color:rgb(183.31235513091087,71.68764486908913,71.68764486908913)'> he</span><span style='background-color:rgb(177.23946779966354,77.76053220033646,77.76053220033646)'> thought</span><span style='background-color:rgb(175.677120834589,79.322879165411,79.322879165411)'> and</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> began</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> to</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> write</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> .</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> they</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> gazed</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> at</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> each</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> other</span><span style='background-color:rgb(170.1841051876545,84.8158948123455,84.8158948123455)'> ,</span><span style='background-color:rgb(170.63374266028404,84.36625733971596,84.36625733971596)'> lost</span><span style='background-color:rgb(163.21449637413025,91.78550362586975,91.78550362586975)'> in</span><span style='background-color:rgb(152.92710319161415,102.07289680838585,102.07289680838585)'> the</span><span style='background-color:rgb(153.15537214279175,101.84462785720825,101.84462785720825)'> rap</span><span style='background-color:rgb(151.91992923617363,103.08007076382637,103.08007076382637)'> ##ture</span><span style='background-color:rgb(148.43567579984665,106.56432420015335,106.56432420015335)'> of</span><span style='background-color:rgb(146.25751867890358,108.74248132109642,108.74248132109642)'> love</span><span style='background-color:rgb(147.16592073440552,107.83407926559448,107.83407926559448)'> based</span><span style='background-color:rgb(144.0287710726261,110.97122892737389,110.97122892737389)'> so</span><span style='background-color:rgb(144.25713881850243,110.74286118149757,110.74286118149757)'> deeply</span><span style='background-color:rgb(138.43578279018402,116.56421720981598,116.56421720981598)'> within</span><span style='background-color:rgb(134.4784927368164,120.5215072631836,120.5215072631836)'> their</span><span style='background-color:rgb(132.756599932909,122.24340006709099,122.24340006709099)'> hearts</span><span style='background-color:rgb(133.03818002343178,121.96181997656822,121.96181997656822)'> that</span><span style='background-color:rgb(132.0277078449726,122.97229215502739,122.97229215502739)'> they</span><span style='background-color:rgb(131.32287606596947,123.67712393403053,123.67712393403053)'> had</span><span style='background-color:rgb(128.75880405306816,126.24119594693184,126.24119594693184)'> never</span><span style='background-color:rgb(117.08342134952545,137.91657865047455,137.91657865047455)'> seen</span><span style='background-color:rgb(108.67020159959793,146.32979840040207,146.32979840040207)'> it</span><span style='background-color:rgb(109.16121125221252,145.83878874778748,145.83878874778748)'> before</span><span style='background-color:rgb(104.92346584796906,150.07653415203094,150.07653415203094)'> .</span><span style='background-color:rgb(105.78840583562851,149.2115941643715,149.2115941643715)'> "</span><span style='background-color:rgb(99.6609154343605,155.3390845656395,155.3390845656395)'> what</span><span style='background-color:rgb(98.0609580874443,156.9390419125557,156.9390419125557)'> about</span><span style='background-color:rgb(94.42385405302048,160.57614594697952,160.57614594697952)'> roger</span><span style='background-color:rgb(90.64600557088852,164.35399442911148,164.35399442911148)'> ?</span><span style='background-color:rgb(84.26684707403183,170.73315292596817,170.73315292596817)'> "</span><span style='background-color:rgb(84.66097712516785,170.33902287483215,170.33902287483215)'> she</span><span style='background-color:rgb(83.72099876403809,171.2790012359619,171.2790012359619)'> asked</span><span style='background-color:rgb(79.19009149074554,175.80990850925446,175.80990850925446)'> ,</span><span style='background-color:rgb(79.8943305015564,175.1056694984436,175.1056694984436)'> knowing</span><span style='background-color:rgb(75.43347626924515,179.56652373075485,179.56652373075485)'> that</span><span style='background-color:rgb(72.25806266069412,182.74193733930588,182.74193733930588)'> the</span><span style='background-color:rgb(68.90556335449219,186.0944366455078,186.0944366455078)'> answer</span><span style='background-color:rgb(67.56203144788742,187.43796855211258,187.43796855211258)'> no</span><span style='background-color:rgb(65.48524528741837,189.51475471258163,189.51475471258163)'> longer</span><span style='background-color:rgb(61.56029909849167,193.43970090150833,193.43970090150833)'> mattered</span><span style='background-color:rgb(57.51620650291443,197.48379349708557,197.48379349708557)'> .</span><span style='background-color:rgb(58.9472097158432,196.0527902841568,196.0527902841568)'> [SEP]</span><span style='background-color:rgb(50.23248374462128,204.76751625537872,204.76751625537872)'> name</span><span style='background-color:rgb(43.587081134319305,211.4129188656807,211.4129188656807)'> few</span><span style='background-color:rgb(36.54069364070892,218.45930635929108,218.45930635929108)'> objects</span><span style='background-color:rgb(32.186583280563354,222.81341671943665,222.81341671943665)'> said</span><span style='background-color:rgb(33.49723935127258,221.50276064872742,221.50276064872742)'> to</span><span style='background-color:rgb(33.46352756023407,221.53647243976593,221.53647243976593)'> be</span><span style='background-color:rgb(35.03389209508896,219.96610790491104,219.96610790491104)'> in</span><span style='background-color:rgb(34.25721377134323,220.74278622865677,220.74278622865677)'> or</span><span style='background-color:rgb(33.22520434856415,221.77479565143585,221.77479565143585)'> on</span><span style='background-color:rgb(31.219535171985626,223.78046482801437,223.78046482801437)'> allan</span><span style='background-color:rgb(27.113247513771057,227.88675248622894,227.88675248622894)'> '</span><span style='background-color:rgb(23.109235167503357,231.89076483249664,231.89076483249664)'> s</span><span style='background-color:rgb(22.15913414955139,232.8408658504486,232.8408658504486)'> desk</span><span style='background-color:rgb(11.813566088676453,243.18643391132355,243.18643391132355)'> |</span><span style='background-color:rgb(7.470596730709076,247.52940326929092,247.52940326929092)'> |</span><span style='background-color:rgb(0.01793503761291504,254.98206496238708,254.98206496238708)'> erase</span><span style='background-color:rgb(0.0,255.0,255.0)'> ##r</span><span style='background-color:rgb(3.2447370886802673,251.75526291131973,251.75526291131973)'> [SEP]</span><span style='background-color:rgb(7.063547372817993,247.936452627182,247.936452627182)'> [PAD]</span><span style='background-color:rgb(9.242107272148132,245.75789272785187,245.75789272785187)'> [PAD]</span><span style='background-color:rgb(15.19971638917923,239.80028361082077,239.80028361082077)'> [PAD]</span><span style='background-color:rgb(23.20815145969391,231.7918485403061,231.7918485403061)'> [PAD]</span><span style='background-color:rgb(24.859345257282257,230.14065474271774,230.14065474271774)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(30.173025727272034,224.82697427272797,224.82697427272797)'> [PAD]</span><span style='background-color:rgb(31.902343332767487,223.0976566672325,223.0976566672325)'> [PAD]</span><span style='background-color:rgb(32.589772045612335,222.41022795438766,222.41022795438766)'> [PAD]</span><span style='background-color:rgb(43.19874197244644,211.80125802755356,211.80125802755356)'> [PAD]</span><span style='background-color:rgb(42.84704804420471,212.1529519557953,212.1529519557953)'> [PAD]</span><span style='background-color:rgb(43.96434009075165,211.03565990924835,211.03565990924835)'> [PAD]</span><span style='background-color:rgb(54.342981576919556,200.65701842308044,200.65701842308044)'> [PAD]</span><span style='background-color:rgb(55.34396946430206,199.65603053569794,199.65603053569794)'> [PAD]</span><span style='background-color:rgb(61.27978295087814,193.72021704912186,193.72021704912186)'> [PAD]</span><span style='background-color:rgb(66.81255966424942,188.18744033575058,188.18744033575058)'> [PAD]</span><span style='background-color:rgb(68.08172196149826,186.91827803850174,186.91827803850174)'> [PAD]</span><span style='background-color:rgb(76.06925815343857,178.93074184656143,178.93074184656143)'> [PAD]</span><span style='background-color:rgb(77.48171836137772,177.51828163862228,177.51828163862228)'> [PAD]</span><span style='background-color:rgb(81.79343819618225,173.20656180381775,173.20656180381775)'> [PAD]</span><span style='background-color:rgb(81.29429697990417,173.70570302009583,173.70570302009583)'> [PAD]</span><span style='background-color:rgb(84.65708613395691,170.3429138660431,170.3429138660431)'> [PAD]</span><span style='background-color:rgb(86.36393934488297,168.63606065511703,168.63606065511703)'> [PAD]</span><span style='background-color:rgb(87.05095767974854,167.94904232025146,167.94904232025146)'> [PAD]</span><span style='background-color:rgb(98.29159051179886,156.70840948820114,156.70840948820114)'> [PAD]</span><span style='background-color:rgb(108.88153105974197,146.11846894025803,146.11846894025803)'> [PAD]</span><span style='background-color:rgb(110.56595027446747,144.43404972553253,144.43404972553253)'> [PAD]</span><span style='background-color:rgb(115.39892613887787,139.60107386112213,139.60107386112213)'> [PAD]</span><span style='background-color:rgb(119.44437146186829,135.5556285381317,135.5556285381317)'> [PAD]</span><span style='background-color:rgb(119.0378388762474,135.9621611237526,135.9621611237526)'> [PAD]</span><span style='background-color:rgb(118.65171879529953,136.34828120470047,136.34828120470047)'> [PAD]</span><span style='background-color:rgb(122.18802183866501,132.811978161335,132.811978161335)'> [PAD]</span><span style='background-color:rgb(127.87712216377258,127.12287783622742,127.12287783622742)'> [PAD]</span><span style='background-color:rgb(132.09413588047028,122.90586411952972,122.90586411952972)'> [PAD]</span><span style='background-color:rgb(131.70822098851204,123.29177901148796,123.29177901148796)'> [PAD]</span><span style='background-color:rgb(142.78557658195496,112.21442341804504,112.21442341804504)'> [PAD]</span><span style='background-color:rgb(145.7580202817917,109.24197971820831,109.24197971820831)'> [PAD]</span><span style='background-color:rgb(145.38764655590057,109.61235344409943,109.61235344409943)'> [PAD]</span><span style='background-color:rgb(147.46017694473267,107.53982305526733,107.53982305526733)'> [PAD]</span><span style='background-color:rgb(150.60817122459412,104.39182877540588,104.39182877540588)'> [PAD]</span><span style='background-color:rgb(149.59796503186226,105.40203496813774,105.40203496813774)'> [PAD]</span><span style='background-color:rgb(151.00825935602188,103.99174064397812,103.99174064397812)'> [PAD]</span><span style='background-color:rgb(150.9089858829975,104.09101411700249,104.09101411700249)'> [PAD]</span><span style='background-color:rgb(156.66658133268356,98.33341866731644,98.33341866731644)'> [PAD]</span><span style='background-color:rgb(160.8556665480137,94.14433345198631,94.14433345198631)'> [PAD]</span><span style='background-color:rgb(163.86039331555367,91.13960668444633,91.13960668444633)'> [PAD]</span><span style='background-color:rgb(165.89181751012802,89.10818248987198,89.10818248987198)'> [PAD]</span><span style='background-color:rgb(174.29807603359222,80.70192396640778,80.70192396640778)'> [PAD]</span><span style='background-color:rgb(181.31078213453293,73.68921786546707,73.68921786546707)'> [PAD]</span><span style='background-color:rgb(185.17715066671371,69.82284933328629,69.82284933328629)'> [PAD]</span><span style='background-color:rgb(192.46727228164673,62.53272771835327,62.53272771835327)'> [PAD]</span><span style='background-color:rgb(192.46727228164673,62.53272771835327,62.53272771835327)'> [PAD]</span><span style='background-color:rgb(192.56711192429066,62.43288807570934,62.43288807570934)'> [PAD]</span><span style='background-color:rgb(193.34369525313377,61.656304746866226,61.656304746866226)'> [PAD]</span><span style='background-color:rgb(195.52235014736652,59.477649852633476,59.477649852633476)'> [PAD]</span><span style='background-color:rgb(199.15968976914883,55.84031023085117,55.84031023085117)'> [PAD]</span><span style='background-color:rgb(206.67867690324783,48.32132309675217,48.32132309675217)'> [PAD]</span><span style='background-color:rgb(212.99743384122849,42.002566158771515,42.002566158771515)'> [PAD]</span><span style='background-color:rgb(211.413184851408,43.586815148591995,43.586815148591995)'> [PAD]</span><span style='background-color:rgb(222.15484745800495,32.84515254199505,32.84515254199505)'> [PAD]</span><span style='background-color:rgb(226.26409325748682,28.73590674251318,28.73590674251318)'> [PAD]</span><span style='background-color:rgb(231.33013151586056,23.669868484139442,23.669868484139442)'> [PAD]</span><span style='background-color:rgb(237.52658251672983,17.47341748327017,17.47341748327017)'> [PAD]</span><span style='background-color:rgb(234.2818435281515,20.718156471848488,20.718156471848488)'> [PAD]</span><span style='background-color:rgb(238.55825375765562,16.44174624234438,16.44174624234438)'> [PAD]</span><span style='background-color:rgb(240.70918913930655,14.290810860693455,14.290810860693455)'> [PAD]</span><span style='background-color:rgb(243.1184745579958,11.881525442004204,11.881525442004204)'> [PAD]</span><span style='background-color:rgb(241.49109598249197,13.50890401750803,13.50890401750803)'> [PAD]</span><span style='background-color:rgb(239.1206482797861,15.87935172021389,15.87935172021389)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(245.8903089724481,9.10969102755189,9.10969102755189)'> [PAD]</span><span style='background-color:rgb(246.3399511948228,8.660048805177212,8.660048805177212)'> [PAD]</span> [PAD]<span style='background-color:rgb(248.46189935691655,6.538100643083453,6.538100643083453)'> [PAD]</span><span style='background-color:rgb(248.68643785826862,6.313562141731381,6.313562141731381)'> [PAD]</span><span style='background-color:rgb(248.68643785826862,6.313562141731381,6.313562141731381)'> [PAD]</span><span style='background-color:rgb(248.32894686609507,6.671053133904934,6.671053133904934)'> [PAD]</span><span style='background-color:rgb(248.68084693327546,6.319153066724539,6.319153066724539)'> [PAD]</span><span style='background-color:rgb(241.96260318160057,13.03739681839943,13.03739681839943)'> [PAD]</span><span style='background-color:rgb(243.6968757212162,11.303124278783798,11.303124278783798)'> [PAD]</span><span style='background-color:rgb(243.92524156719446,11.074758432805538,11.074758432805538)'> [PAD]</span><span style='background-color:rgb(243.14786028116941,11.852139718830585,11.852139718830585)'> [PAD]</span><span style='background-color:rgb(245.51907364279032,9.480926357209682,9.480926357209682)'> [PAD]</span><span style='background-color:rgb(243.37163312360644,11.628366876393557,11.628366876393557)'> [PAD]</span><span style='background-color:rgb(243.6532132141292,11.34678678587079,11.34678678587079)'> [PAD]</span><span style='background-color:rgb(242.38465603441,12.61534396559,12.61534396559)'> [PAD]</span><span style='background-color:rgb(246.19558364152908,8.804416358470917,8.804416358470917)'> [PAD]</span><span style='background-color:rgb(248.3747396338731,6.625260366126895,6.625260366126895)'> [PAD]</span><span style='background-color:rgb(244.70112478360534,10.298875216394663,10.298875216394663)'> [PAD]</span><span style='background-color:rgb(240.34427667036653,14.655723329633474,14.655723329633474)'> [PAD]</span><span style='background-color:rgb(240.16889233142138,14.831107668578625,14.831107668578625)'> [PAD]</span><span style='background-color:rgb(235.80146178603172,19.198538213968277,19.198538213968277)'> [PAD]</span><span style='background-color:rgb(235.97890846431255,19.021091535687447,19.021091535687447)'> [PAD]</span><span style='background-color:rgb(239.95377257466316,15.046227425336838,15.046227425336838)'> [PAD]</span><span style='background-color:rgb(242.46070040389895,12.539299596101046,12.539299596101046)'> [PAD]</span><span style='background-color:rgb(238.49142864346504,16.508571356534958,16.508571356534958)'> [PAD]</span><span style='background-color:rgb(234.79115679860115,20.20884320139885,20.20884320139885)'> [PAD]</span><span style='background-color:rgb(238.99168889969587,16.008311100304127,16.008311100304127)'> [PAD]</span><span style='background-color:rgb(240.51986049860716,14.480139501392841,14.480139501392841)'> [PAD]</span><span style='background-color:rgb(232.5617480278015,22.438251972198486,22.438251972198486)'> [PAD]</span><span style='background-color:rgb(231.38510886579752,23.61489113420248,23.61489113420248)'> [PAD]</span><span style='background-color:rgb(231.36873554438353,23.631264455616474,23.631264455616474)'> [PAD]</span><span style='background-color:rgb(238.75077802687883,16.249221973121166,16.249221973121166)'> [PAD]</span><span style='background-color:rgb(244.31727603077888,10.682723969221115,10.682723969221115)'> [PAD]</span><span style='background-color:rgb(246.9949296489358,8.005070351064205,8.005070351064205)'> [PAD]</span><span style='background-color:rgb(245.28208890929818,9.717911090701818,9.717911090701818)'> [PAD]</span><span style='background-color:rgb(247.557422965765,7.442577034235001,7.442577034235001)'> [PAD]</span><span style='background-color:rgb(249.73667917773128,5.263320822268724,5.263320822268724)'> [PAD]</span><span style='background-color:rgb(247.58893894962966,7.411061050370336,7.411061050370336)'> [PAD]</span><span style='background-color:rgb(249.92986793629825,5.070132063701749,5.070132063701749)'> [PAD]</span><span style='background-color:rgb(251.8863827129826,3.113617287017405,3.113617287017405)'> [PAD]</span><span style='background-color:rgb(245.53621262311935,9.463787376880646,9.463787376880646)'> [PAD]</span><span style='background-color:rgb(251.5135500440374,3.4864499559625983,3.4864499559625983)'> [PAD]</span><span style='background-color:rgb(249.58359251730144,5.41640748269856,5.41640748269856)'> [PAD]</span><span style='background-color:rgb(244.18535565957427,10.81464434042573,10.81464434042573)'> [PAD]</span><span style='background-color:rgb(244.21913489699364,10.780865103006363,10.780865103006363)'> [PAD]</span><span style='background-color:rgb(244.21913489699364,10.780865103006363,10.780865103006363)'> [PAD]</span><span style='background-color:rgb(247.64258589595556,7.357414104044437,7.357414104044437)'> [PAD]</span><span style='background-color:rgb(244.214309155941,10.78569084405899,10.78569084405899)'> [PAD]</span><span style='background-color:rgb(240.50634937360883,14.493650626391172,14.493650626391172)'> [PAD]</span><span style='background-color:rgb(236.17559161037207,18.824408389627934,18.824408389627934)'> [PAD]</span><span style='background-color:rgb(235.51625479012728,19.483745209872723,19.483745209872723)'> [PAD]</span><span style='background-color:rgb(243.42963986098766,11.570360139012337,11.570360139012337)'> [PAD]</span><span style='background-color:rgb(242.73203054443002,12.267969455569983,12.267969455569983)'> [PAD]</span><span style='background-color:rgb(240.55353999137878,14.446460008621216,14.446460008621216)'> [PAD]</span><span style='background-color:rgb(244.83823662623763,10.16176337376237,10.16176337376237)'> [PAD]</span><span style='background-color:rgb(247.51046556048095,7.489534439519048,7.489534439519048)'> [PAD]</span><span style='background-color:rgb(247.51046556048095,7.489534439519048,7.489534439519048)'> [PAD]</span><span style='background-color:rgb(245.87057378143072,9.129426218569279,9.129426218569279)'> [PAD]</span><span style='background-color:rgb(245.62370482832193,9.376295171678066,9.376295171678066)'> [PAD]</span><span style='background-color:rgb(243.44514777883887,11.554852221161127,11.554852221161127)'> [PAD]</span><span style='background-color:rgb(238.9019339159131,16.098066084086895,16.098066084086895)'> [PAD]</span><span style='background-color:rgb(239.2381921224296,15.76180787757039,15.76180787757039)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span>

In [87]:
rise_expln.attributions[0][:100]

tensor([143.8457, 143.8457, 143.8457, 143.8457, 143.8457, 143.8457, 143.8457,
        143.8457, 143.8457, 143.7377, 143.9612, 143.1321, 143.2646, 143.1600,
        142.5935, 144.5916, 144.7318, 143.5122, 145.8715, 145.9230, 142.8716,
        145.6129, 146.7980, 146.8977, 147.3217, 147.7874, 148.5657, 148.7686,
        150.7664, 151.6030, 151.4199, 151.7917, 149.9019, 150.1675, 149.8063,
        149.1363, 149.1290, 149.0445, 153.3457, 152.1779, 151.1791, 154.8050,
        155.1800, 155.5230, 156.8745, 156.8290, 155.1501, 154.0721, 155.3260,
        154.5323, 156.3637, 159.0486, 157.8299, 159.3602, 160.3593, 158.5566,
        157.3667, 157.4178, 156.3882, 153.9164, 154.0369, 153.3644, 150.9707,
        149.9720, 153.0543, 152.7269, 151.2212, 153.4665, 154.4657, 152.5734,
        149.5909, 149.6595, 149.8358, 149.8358, 149.8358, 149.8358, 149.8358,
        149.8358, 149.8358, 149.8358, 149.8358, 148.8377, 150.0430, 150.4264,
        150.4249, 150.6325, 150.5783, 150.6839, 148.8859, 149.30

In [89]:
rise_expln.explainer_output.shape

torch.Size([2, 2, 512])

In [90]:
rise_expln.explainer_output

tensor([[[143.8457, 143.8457, 143.8457,  ..., 143.6499, 143.6499, 143.6499],
         [108.1543, 108.1543, 108.1543,  ..., 105.3501, 105.3501, 105.3501]],

        [[143.6210, 143.6210, 143.6210,  ..., 143.3065, 143.3065, 143.3065],
         [108.3790, 108.3790, 108.3790,  ..., 105.6935, 105.6935, 105.6935]]],
       device='cuda:0')

In [92]:
j = 0
grouped_attrs_aggr = rise_expln.explainer_output[j].mean(-1)
aggr_pred = torch.argmax(grouped_attrs_aggr)
aggr_pred

tensor(0, device='cuda:0')

## IntGrad

In [79]:
from exlib.explainers.common import *

def explain_text_cls_with_intgrad(model, x, label,
                             x0 = None,
                             num_steps = 32,
                             progress_bar = False,
                             x_kwargs = {},
                             mask_combine=None):
    """
    Explain a classification model with Integrated Gradients.
    """
    assert x.size(0) == len(label)

    # Default baseline is zeros
    x0 = torch.zeros_like(x) if x0 is None else x0

    step_size = 1 / num_steps
    intg = torch.zeros_like(x).float()

    pbar = tqdm(range(num_steps)) if progress_bar else range(num_steps)
    for k in pbar:
        ak = k * step_size
        
        if mask_combine:
            mask = ak * torch.ones_like(x[0]).float()
            xk = mask_combine(x, mask).squeeze(1)
            xk.requires_grad_()
            y = model(inputs_embeds=xk, **x_kwargs)
        else:
            # xk = x0 + ak * (x - x0)
            if ak >= 0.5:
                xk = x
            else:
                xk = x0
            import pdb; pdb.set_trace()
            xk.requires_grad_()
            y = model(xk, **x_kwargs)

        loss = 0.0
        for i, l in enumerate(label):
            loss += y[i, l]

        loss.backward()
        if mask_combine is not None:
            intg += xk.grad.sum(-1) * step_size
        else:
            intg += xk.grad * step_size

    return FeatureAttrOutput(intg, {})


class IntGradTextCls(FeatureAttrMethod):
    """ Image classification with integrated gradients
    """
    def __init__(self, model, mask_combine='default', projection_layer=None):
        super().__init__(model)
        if mask_combine == 'default':
            def mask_combine(inputs, masks):
                with torch.no_grad():
                    inputs_embed = projection_layer(inputs)
                    mask_embed = projection_layer(torch.tensor([0]).int().to(inputs.device))
                    masked_inputs_embeds = inputs_embed.unsqueeze(1) * masks.unsqueeze(-1) + \
                            mask_embed.view(1,1,1,-1) * (1 - masks.unsqueeze(-1))
                return masked_inputs_embeds
        self.mask_combine = mask_combine

    def forward(self, x, t, **kwargs):
        if not isinstance(t, torch.Tensor):
            t = torch.tensor(t)

        with torch.enable_grad():
            return explain_text_cls_with_intgrad(self.model, x, t, 
                                                 mask_combine=self.mask_combine,
                                                 **kwargs)

In [80]:
from exlib.explainers.intgrad import IntGradTextCls

intgrad_explainer = IntGradTextCls(original_model2, projection_layer=projection_layer).to(device)
intgrad_expln = intgrad_explainer(inputs, labels, x_kwargs=kwargs)

In [81]:
intgrad_expln.attributions.shape

torch.Size([2, 512])

In [82]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks, normalize=True):
    if normalize:
        masks_max = masks.max(-1).values
        masks_min = masks.min(-1).values
        # import pdb; pdb.set_trace()
        masks = (masks - masks_min) / (masks_max - masks_min)
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

tokens = processor.convert_ids_to_tokens(inputs[0])
# tokens = shap_expln.explainer_output.data[0]
printmd(show_attrs(tokens, intgrad_expln.attributions[0]))

<span style='background-color:rgb(144.50095653533936,110.49904346466064,110.49904346466064)'> [CLS]</span><span style='background-color:rgb(132.0989692211151,122.90103077888489,122.90103077888489)'> allan</span><span style='background-color:rgb(134.04495880007744,120.95504119992256,120.95504119992256)'> sat</span><span style='background-color:rgb(136.14323660731316,118.85676339268684,118.85676339268684)'> down</span><span style='background-color:rgb(137.06120654940605,117.93879345059395,117.93879345059395)'> at</span><span style='background-color:rgb(130.0081466138363,124.99185338616371,124.99185338616371)'> his</span><span style='background-color:rgb(133.96893247961998,121.03106752038002,121.03106752038002)'> desk</span><span style='background-color:rgb(129.3670678138733,125.63293218612671,125.63293218612671)'> and</span><span style='background-color:rgb(132.7328284084797,122.26717159152031,122.26717159152031)'> pulled</span><span style='background-color:rgb(134.1080506145954,120.89194938540459,120.89194938540459)'> the</span><span style='background-color:rgb(133.95312532782555,121.04687467217445,121.04687467217445)'> chair</span><span style='background-color:rgb(133.92766669392586,121.07233330607414,121.07233330607414)'> in</span><span style='background-color:rgb(133.9499182999134,121.0500817000866,121.0500817000866)'> close</span><span style='background-color:rgb(131.36246994137764,123.63753005862236,123.63753005862236)'> .</span><span style='background-color:rgb(132.32935085892677,122.67064914107323,122.67064914107323)'> opening</span><span style='background-color:rgb(134.33515682816505,120.66484317183495,120.66484317183495)'> a</span><span style='background-color:rgb(132.4335716664791,122.56642833352089,122.56642833352089)'> side</span><span style='background-color:rgb(128.74072462320328,126.25927537679672,126.25927537679672)'> drawer</span><span style='background-color:rgb(129.97862979769707,125.02137020230293,125.02137020230293)'> ,</span><span style='background-color:rgb(132.79221922159195,122.20778077840805,122.20778077840805)'> he</span><span style='background-color:rgb(131.6463679075241,123.35363209247589,123.35363209247589)'> took</span><span style='background-color:rgb(130.3556987643242,124.64430123567581,124.64430123567581)'> out</span><span style='background-color:rgb(129.25416067242622,125.74583932757378,125.74583932757378)'> a</span><span style='background-color:rgb(134.20214876532555,120.79785123467445,120.79785123467445)'> piece</span><span style='background-color:rgb(129.80765417218208,125.19234582781792,125.19234582781792)'> of</span><span style='background-color:rgb(132.20857053995132,122.79142946004868,122.79142946004868)'> paper</span><span style='background-color:rgb(135.06886705756187,119.93113294243813,119.93113294243813)'> and</span><span style='background-color:rgb(133.08496311306953,121.91503688693047,121.91503688693047)'> his</span><span style='background-color:rgb(133.75678226351738,121.24321773648262,121.24321773648262)'> ink</span><span style='background-color:rgb(133.7756596505642,121.2243403494358,121.2243403494358)'> ##pot</span><span style='background-color:rgb(134.6235689520836,120.37643104791641,120.37643104791641)'> .</span><span style='background-color:rgb(135.5117864906788,119.48821350932121,119.48821350932121)'> after</span><span style='background-color:rgb(132.23716020584106,122.76283979415894,122.76283979415894)'> filling</span><span style='background-color:rgb(134.2694203555584,120.7305796444416,120.7305796444416)'> his</span><span style='background-color:rgb(130.6332890689373,124.3667109310627,124.3667109310627)'> pen</span><span style='background-color:rgb(130.93391373753548,124.06608626246452,124.06608626246452)'> ,</span><span style='background-color:rgb(133.3642253279686,121.6357746720314,121.6357746720314)'> allan</span><span style='background-color:rgb(130.6434117257595,124.3565882742405,124.3565882742405)'> looked</span><span style='background-color:rgb(132.90014103055,122.09985896945,122.09985896945)'> at</span><span style='background-color:rgb(135.57951405644417,119.42048594355583,119.42048594355583)'> his</span><span style='background-color:rgb(133.40020939707756,121.59979060292244,121.59979060292244)'> paper</span><span style='background-color:rgb(131.76960289478302,123.23039710521698,123.23039710521698)'> in</span><span style='background-color:rgb(133.53413701057434,121.46586298942566,121.46586298942566)'> the</span><span style='background-color:rgb(132.54353016614914,122.45646983385086,122.45646983385086)'> orange</span><span style='background-color:rgb(131.68466985225677,123.31533014774323,123.31533014774323)'> glow</span><span style='background-color:rgb(131.87836065888405,123.12163934111595,123.12163934111595)'> from</span><span style='background-color:rgb(136.04892566800117,118.95107433199883,118.95107433199883)'> the</span><span style='background-color:rgb(129.82375010848045,125.17624989151955,125.17624989151955)'> lantern</span><span style='background-color:rgb(132.27803081274033,122.72196918725967,122.72196918725967)'> set</span><span style='background-color:rgb(131.9969978928566,123.0030021071434,123.0030021071434)'> back</span><span style='background-color:rgb(131.11679032444954,123.88320967555046,123.88320967555046)'> in</span><span style='background-color:rgb(130.8625991642475,124.13740083575249,124.13740083575249)'> the</span><span style='background-color:rgb(131.66409015655518,123.33590984344482,123.33590984344482)'> desk</span><span style='background-color:rgb(133.6851941049099,121.3148058950901,121.3148058950901)'> '</span><span style='background-color:rgb(135.07064536213875,119.92935463786125,119.92935463786125)'> s</span><span style='background-color:rgb(128.94370213150978,126.05629786849022,126.05629786849022)'> right</span><span style='background-color:rgb(130.16802683472633,124.83197316527367,124.83197316527367)'> -</span><span style='background-color:rgb(132.1432976424694,122.8567023575306,122.8567023575306)'> hand</span><span style='background-color:rgb(131.40995979309082,123.59004020690918,123.59004020690918)'> corner</span><span style='background-color:rgb(136.63932278752327,118.36067721247673,118.36067721247673)'> .</span><span style='background-color:rgb(130.72190031409264,124.27809968590736,124.27809968590736)'> his</span><span style='background-color:rgb(132.33286947011948,122.66713052988052,122.66713052988052)'> pen</span><span style='background-color:rgb(128.76153230667114,126.23846769332886,126.23846769332886)'> cast</span><span style='background-color:rgb(126.10863626003265,128.89136373996735,128.89136373996735)'> a</span><span style='background-color:rgb(133.6700101196766,121.32998988032341,121.32998988032341)'> forbid</span><span style='background-color:rgb(131.6218440234661,123.37815597653389,123.37815597653389)'> ##ding</span><span style='background-color:rgb(134.3740439414978,120.6259560585022,120.6259560585022)'> line</span><span style='background-color:rgb(132.88569420576096,122.11430579423904,122.11430579423904)'> of</span><span style='background-color:rgb(133.07834386825562,121.92165613174438,121.92165613174438)'> shadow</span><span style='background-color:rgb(132.71240070462227,122.28759929537773,122.28759929537773)'> slant</span><span style='background-color:rgb(132.42502212524414,122.57497787475586,122.57497787475586)'> ##ing</span><span style='background-color:rgb(131.60954028367996,123.39045971632004,123.39045971632004)'> across</span><span style='background-color:rgb(141.18972301483154,113.81027698516846,113.81027698516846)'> the</span><span style='background-color:rgb(136.10852167010307,118.89147832989693,118.89147832989693)'> page</span><span style='background-color:rgb(132.42572888731956,122.57427111268044,122.57427111268044)'> ,</span><span style='background-color:rgb(132.40804463624954,122.59195536375046,122.59195536375046)'> echoing</span><span style='background-color:rgb(131.8108230829239,123.18917691707611,123.18917691707611)'> the</span><span style='background-color:rgb(130.5015653371811,124.49843466281891,124.49843466281891)'> ink</span><span style='background-color:rgb(133.0512437224388,121.94875627756119,121.94875627756119)'> ##y</span><span style='background-color:rgb(135.38370296359062,119.61629703640938,119.61629703640938)'> darkness</span><span style='background-color:rgb(131.84945940971375,123.15054059028625,123.15054059028625)'> crouch</span><span style='background-color:rgb(132.49707385897636,122.50292614102364,122.50292614102364)'> ##ing</span><span style='background-color:rgb(134.95611190795898,120.04388809204102,120.04388809204102)'> in</span><span style='background-color:rgb(130.02491131424904,124.97508868575096,124.97508868575096)'> the</span><span style='background-color:rgb(132.95976743102074,122.04023256897926,122.04023256897926)'> edges</span><span style='background-color:rgb(132.78088063001633,122.21911936998367,122.21911936998367)'> of</span><span style='background-color:rgb(131.76053658127785,123.23946341872215,123.23946341872215)'> the</span><span style='background-color:rgb(132.2156685590744,122.7843314409256,122.7843314409256)'> lantern</span><span style='background-color:rgb(131.87594398856163,123.12405601143837,123.12405601143837)'> '</span><span style='background-color:rgb(129.80236485600471,125.19763514399529,125.19763514399529)'> s</span><span style='background-color:rgb(135.6096464395523,119.39035356044769,119.39035356044769)'> struggling</span><span style='background-color:rgb(132.6569464802742,122.3430535197258,122.3430535197258)'> glow</span><span style='background-color:rgb(132.46335446834564,122.53664553165436,122.53664553165436)'> .</span><span style='background-color:rgb(134.61318790912628,120.38681209087372,120.38681209087372)'> the</span><span style='background-color:rgb(132.7177508175373,122.28224918246269,122.28224918246269)'> only</span><span style='background-color:rgb(134.28724139928818,120.71275860071182,120.71275860071182)'> other</span><span style='background-color:rgb(130.28572171926498,124.71427828073502,124.71427828073502)'> illumination</span><span style='background-color:rgb(131.0872583091259,123.9127416908741,123.9127416908741)'> came</span><span style='background-color:rgb(133.99908006191254,121.00091993808746,121.00091993808746)'> from</span><span style='background-color:rgb(130.3141137957573,124.6858862042427,124.6858862042427)'> a</span><span style='background-color:rgb(132.40248173475266,122.59751826524734,122.59751826524734)'> lu</span><span style='background-color:rgb(126.8193045258522,128.1806954741478,128.1806954741478)'> ##rid</span><span style='background-color:rgb(128.57219606637955,126.42780393362045,126.42780393362045)'> moonlight</span><span style='background-color:rgb(130.96448689699173,124.03551310300827,124.03551310300827)'> filtered</span><span style='background-color:rgb(129.19280156493187,125.80719843506813,125.80719843506813)'> through</span><span style='background-color:rgb(137.8474147617817,117.15258523821831,117.15258523821831)'> thin</span><span style='background-color:rgb(133.92716512084007,121.07283487915993,121.07283487915993)'> branches</span><span style='background-color:rgb(137.64791786670685,117.35208213329315,117.35208213329315)'> and</span><span style='background-color:rgb(133.43407317996025,121.56592682003975,121.56592682003975)'> clouds</span><span style='background-color:rgb(137.60823279619217,117.39176720380783,117.39176720380783)'> ,</span><span style='background-color:rgb(132.9293918609619,122.07060813903809,122.07060813903809)'> casting</span><span style='background-color:rgb(130.53174331784248,124.46825668215752,124.46825668215752)'> its</span><span style='background-color:rgb(133.6068271100521,121.39317288994789,121.39317288994789)'> bone</span><span style='background-color:rgb(132.138152718544,122.861847281456,122.861847281456)'> -</span><span style='background-color:rgb(134.43108648061752,120.56891351938248,120.56891351938248)'> pale</span><span style='background-color:rgb(129.45682659745216,125.54317340254784,125.54317340254784)'> glow</span><span style='background-color:rgb(134.9840708076954,120.01592919230461,120.01592919230461)'> onto</span><span style='background-color:rgb(131.62857726216316,123.37142273783684,123.37142273783684)'> the</span><span style='background-color:rgb(133.55128929018974,121.44871070981026,121.44871070981026)'> pine</span><span style='background-color:rgb(131.5256331861019,123.47436681389809,123.47436681389809)'> floor</span><span style='background-color:rgb(132.15037286281586,122.84962713718414,122.84962713718414)'> ##boards</span><span style='background-color:rgb(135.9889116883278,119.01108831167221,119.01108831167221)'> .</span><span style='background-color:rgb(129.24618110060692,125.75381889939308,125.75381889939308)'> allan</span><span style='background-color:rgb(131.47058174014091,123.52941825985909,123.52941825985909)'> unfolded</span><span style='background-color:rgb(128.6462540924549,126.35374590754509,126.35374590754509)'> another</span><span style='background-color:rgb(131.8999207019806,123.10007929801941,123.10007929801941)'> page</span><span style='background-color:rgb(131.53143927454948,123.46856072545052,123.46856072545052)'> ,</span><span style='background-color:rgb(131.39980673789978,123.60019326210022,123.60019326210022)'> this</span><span style='background-color:rgb(136.1343678832054,118.86563211679459,118.86563211679459)'> one</span><span style='background-color:rgb(131.1370812356472,123.8629187643528,123.8629187643528)'> crowded</span><span style='background-color:rgb(131.82556629180908,123.17443370819092,123.17443370819092)'> with</span><span style='background-color:rgb(132.96952530741692,122.03047469258308,122.03047469258308)'> ranks</span><span style='background-color:rgb(139.12917718291283,115.87082281708717,115.87082281708717)'> of</span><span style='background-color:rgb(138.56072008609772,116.43927991390228,116.43927991390228)'> letters</span><span style='background-color:rgb(128.0450503528118,126.95494964718819,126.95494964718819)'> in</span><span style='background-color:rgb(130.51590576767921,124.48409423232079,124.48409423232079)'> tight</span><span style='background-color:rgb(131.5129266679287,123.4870733320713,123.4870733320713)'> formation</span><span style='background-color:rgb(129.54650178551674,125.45349821448326,125.45349821448326)'> from</span><span style='background-color:rgb(137.9438155889511,117.05618441104889,117.05618441104889)'> left</span><span style='background-color:rgb(136.49572849273682,118.50427150726318,118.50427150726318)'> to</span><span style='background-color:rgb(131.42978712916374,123.57021287083626,123.57021287083626)'> right</span><span style='background-color:rgb(130.02261623740196,124.97738376259804,124.97738376259804)'> .</span><span style='background-color:rgb(129.44893062114716,125.55106937885284,125.55106937885284)'> the</span><span style='background-color:rgb(127.09307223558426,127.90692776441574,127.90692776441574)'> lines</span><span style='background-color:rgb(125.79323798418045,129.20676201581955,129.20676201581955)'> of</span><span style='background-color:rgb(132.68273949623108,122.31726050376892,122.31726050376892)'> letters</span><span style='background-color:rgb(132.40582555532455,122.59417444467545,122.59417444467545)'> stepped</span><span style='background-color:rgb(130.1944962143898,124.8055037856102,124.8055037856102)'> into</span><span style='background-color:rgb(144.69441175460815,110.30558824539185,110.30558824539185)'> their</span><span style='background-color:rgb(125.81538319587708,129.18461680412292,129.18461680412292)'> divisions</span><span style='background-color:rgb(125.71724206209183,129.28275793790817,129.28275793790817)'> ,</span><span style='background-color:rgb(129.50560837984085,125.49439162015915,125.49439162015915)'> in</span><span style='background-color:rgb(134.69764977693558,120.30235022306442,120.30235022306442)'> the</span><span style='background-color:rgb(129.68232169747353,125.31767830252647,125.31767830252647)'> shape</span><span style='background-color:rgb(136.04170605540276,118.95829394459724,118.95829394459724)'> of</span><span style='background-color:rgb(136.11183509230614,118.88816490769386,118.88816490769386)'> a</span><span style='background-color:rgb(133.04527804255486,121.95472195744514,121.95472195744514)'> story</span><span style='background-color:rgb(132.5428006052971,122.45719939470291,122.45719939470291)'> '</span><span style='background-color:rgb(134.25273925065994,120.74726074934006,120.74726074934006)'> s</span><span style='background-color:rgb(128.6859391629696,126.31406083703041,126.31406083703041)'> outline</span><span style='background-color:rgb(132.9425923526287,122.05740764737129,122.05740764737129)'> :</span><span style='background-color:rgb(138.39187234640121,116.60812765359879,116.60812765359879)'> the</span><span style='background-color:rgb(132.04907029867172,122.95092970132828,122.95092970132828)'> loose</span><span style='background-color:rgb(133.3613982796669,121.6386017203331,121.6386017203331)'> ,</span><span style='background-color:rgb(134.5080779492855,120.49192205071449,120.49192205071449)'> dry</span><span style='background-color:rgb(132.22130745649338,122.77869254350662,122.77869254350662)'> skeleton</span><span style='background-color:rgb(126.33010357618332,128.66989642381668,128.66989642381668)'> of</span><span style='background-color:rgb(134.90585580468178,120.09414419531822,120.09414419531822)'> a</span><span style='background-color:rgb(125.69261938333511,129.3073806166649,129.3073806166649)'> tale</span><span style='background-color:rgb(135.72417229413986,119.27582770586014,119.27582770586014)'> lay</span><span style='background-color:rgb(131.8586777150631,123.1413222849369,123.1413222849369)'> exposed</span><span style='background-color:rgb(129.77450475096703,125.22549524903297,125.22549524903297)'> beneath</span><span style='background-color:rgb(130.80403670668602,124.19596329331398,124.19596329331398)'> their</span><span style='background-color:rgb(131.655533015728,123.344466984272,123.344466984272)'> feet</span><span style='background-color:rgb(137.13094800710678,117.86905199289322,117.86905199289322)'> ,</span><span style='background-color:rgb(128.24213057756424,126.75786942243576,126.75786942243576)'> awaiting</span><span style='background-color:rgb(133.78297805786133,121.21702194213867,121.21702194213867)'> tend</span><span style='background-color:rgb(128.78415629267693,126.21584370732307,126.21584370732307)'> ##ons</span><span style='background-color:rgb(134.38102796673775,120.61897203326225,120.61897203326225)'> ,</span><span style='background-color:rgb(131.24334633350372,123.75665366649628,123.75665366649628)'> muscle</span><span style='background-color:rgb(141.3388878107071,113.66111218929291,113.66111218929291)'> and</span><span style='background-color:rgb(130.1298388838768,124.8701611161232,124.8701611161232)'> blushing</span><span style='background-color:rgb(136.2596395611763,118.7403604388237,118.7403604388237)'> skin</span><span style='background-color:rgb(129.07530426979065,125.92469573020935,125.92469573020935)'> .</span><span style='background-color:rgb(128.2886552810669,126.7113447189331,126.7113447189331)'> allan</span><span style='background-color:rgb(130.6482830643654,124.35171693563461,124.35171693563461)'> reviewed</span><span style='background-color:rgb(131.84080347418785,123.15919652581215,123.15919652581215)'> the</span><span style='background-color:rgb(135.0533562898636,119.94664371013641,119.94664371013641)'> troops</span><span style='background-color:rgb(135.77743023633957,119.22256976366043,119.22256976366043)'> ,</span><span style='background-color:rgb(132.1288812160492,122.8711187839508,122.8711187839508)'> all</span><span style='background-color:rgb(133.67358952760696,121.32641047239304,121.32641047239304)'> prepared</span><span style='background-color:rgb(131.61178216338158,123.38821783661842,123.38821783661842)'> to</span><span style='background-color:rgb(133.71766716241837,121.28233283758163,121.28233283758163)'> di</span><span style='background-color:rgb(132.0082224905491,122.99177750945091,122.99177750945091)'> ##se</span><span style='background-color:rgb(133.40486034750938,121.59513965249062,121.59513965249062)'> ##mba</span><span style='background-color:rgb(133.2115875184536,121.7884124815464,121.7884124815464)'> ##rk</span><span style='background-color:rgb(131.06917887926102,123.93082112073898,123.93082112073898)'> ,</span><span style='background-color:rgb(132.19829589128494,122.80170410871506,122.80170410871506)'> their</span><span style='background-color:rgb(133.46827894449234,121.53172105550766,121.53172105550766)'> task</span><span style='background-color:rgb(132.85390511155128,122.14609488844872,122.14609488844872)'> to</span><span style='background-color:rgb(131.92148834466934,123.07851165533066,123.07851165533066)'> form</span><span style='background-color:rgb(133.96383315324783,121.03616684675217,121.03616684675217)'> the</span><span style='background-color:rgb(132.9094885289669,122.0905114710331,122.0905114710331)'> tale</span><span style='background-color:rgb(133.51079866290092,121.48920133709908,121.48920133709908)'> of</span><span style='background-color:rgb(133.58291879296303,121.41708120703697,121.41708120703697)'> a</span><span style='background-color:rgb(132.50656574964523,122.49343425035477,122.49343425035477)'> young</span><span style='background-color:rgb(133.0388107895851,121.96118921041489,121.96118921041489)'> man</span><span style='background-color:rgb(133.5865893959999,121.41341060400009,121.41341060400009)'> returning</span><span style='background-color:rgb(136.46173551678658,118.53826448321342,118.53826448321342)'> home</span><span style='background-color:rgb(135.4451684653759,119.5548315346241,119.5548315346241)'> from</span><span style='background-color:rgb(133.58034253120422,121.41965746879578,121.41965746879578)'> life</span><span style='background-color:rgb(135.02684131264687,119.97315868735313,119.97315868735313)'> abroad</span><span style='background-color:rgb(134.42861661314964,120.57138338685036,120.57138338685036)'> to</span><span style='background-color:rgb(129.8851092159748,125.11489078402519,125.11489078402519)'> find</span><span style='background-color:rgb(130.72065398097038,124.27934601902962,124.27934601902962)'> his</span><span style='background-color:rgb(131.9753086566925,123.0246913433075,123.0246913433075)'> childhood</span><span style='background-color:rgb(131.49563759565353,123.50436240434647,123.50436240434647)'> friend</span><span style='background-color:rgb(131.19533210992813,123.80466789007187,123.80466789007187)'> a</span><span style='background-color:rgb(131.92603290081024,123.07396709918976,123.07396709918976)'> bride</span><span style='background-color:rgb(132.37547278404236,122.62452721595764,122.62452721595764)'> to</span><span style='background-color:rgb(135.66532105207443,119.33467894792557,119.33467894792557)'> -</span><span style='background-color:rgb(132.4174529314041,122.58254706859589,122.58254706859589)'> be</span><span style='background-color:rgb(131.2878343462944,123.7121656537056,123.7121656537056)'> ,</span><span style='background-color:rgb(132.26261124014854,122.73738875985146,122.73738875985146)'> thus</span><span style='background-color:rgb(133.01875546574593,121.98124453425407,121.98124453425407)'> upset</span><span style='background-color:rgb(132.55157813429832,122.44842186570168,122.44842186570168)'> ##ting</span><span style='background-color:rgb(127.3177769780159,127.6822230219841,127.6822230219841)'> the</span><span style='background-color:rgb(134.24294337630272,120.75705662369728,120.75705662369728)'> apple</span><span style='background-color:rgb(129.43829119205475,125.56170880794525,125.56170880794525)'> cart</span><span style='background-color:rgb(128.35900470614433,126.64099529385567,126.64099529385567)'> of</span><span style='background-color:rgb(135.8044771850109,119.19552281498909,119.19552281498909)'> his</span><span style='background-color:rgb(124.5052894949913,130.4947105050087,130.4947105050087)'> life</span><span style='background-color:rgb(133.37592110037804,121.62407889962196,121.62407889962196)'> '</span><span style='background-color:rgb(128.28543305397034,126.71456694602966,126.71456694602966)'> s</span><span style='background-color:rgb(134.20111522078514,120.79888477921486,120.79888477921486)'> plan</span><span style='background-color:rgb(127.60661467909813,127.39338532090187,127.39338532090187)'> ,</span><span style='background-color:rgb(137.63917833566666,117.36082166433334,117.36082166433334)'> clarified</span><span style='background-color:rgb(132.17766299843788,122.82233700156212,122.82233700156212)'> –</span><span style='background-color:rgb(132.56169319152832,122.43830680847168,122.43830680847168)'> of</span><span style='background-color:rgb(131.3668929040432,123.6331070959568,123.6331070959568)'> course</span><span style='background-color:rgb(128.2727113366127,126.7272886633873,126.7272886633873)'> –</span><span style='background-color:rgb(134.51582953333855,120.48417046666145,120.48417046666145)'> by</span><span style='background-color:rgb(137.71594181656837,117.28405818343163,117.28405818343163)'> his</span><span style='background-color:rgb(135.08615612983704,119.91384387016296,119.91384387016296)'> very</span><span style='background-color:rgb(130.70252135396004,124.29747864603996,124.29747864603996)'> time</span><span style='background-color:rgb(133.48500564694405,121.51499435305595,121.51499435305595)'> away</span><span style='background-color:rgb(138.2489924132824,116.7510075867176,116.7510075867176)'> from</span><span style='background-color:rgb(130.68682059645653,124.31317940354347,124.31317940354347)'> her</span><span style='background-color:rgb(122.33151733875275,132.66848266124725,132.66848266124725)'> he</span><span style='background-color:rgb(126.65910512208939,128.3408948779106,128.3408948779106)'> loved</span><span style='background-color:rgb(141.84273317456245,113.15726682543755,113.15726682543755)'> best</span><span style='background-color:rgb(130.60313388705254,124.39686611294746,124.39686611294746)'> .</span><span style='background-color:rgb(133.5105250775814,121.4894749224186,121.4894749224186)'> although</span><span style='background-color:rgb(127.26514220237732,127.73485779762268,127.73485779762268)'> the</span><span style='background-color:rgb(136.1700403690338,118.82995963096619,118.82995963096619)'> concept</span><span style='background-color:rgb(128.54692742228508,126.45307257771492,126.45307257771492)'> was</span><span style='background-color:rgb(131.86535775661469,123.13464224338531,123.13464224338531)'> a</span><span style='background-color:rgb(135.1399004459381,119.86009955406189,119.86009955406189)'> simple</span><span style='background-color:rgb(132.67093732953072,122.32906267046928,122.32906267046928)'> one</span><span style='background-color:rgb(137.09942489862442,117.90057510137558,117.90057510137558)'> ,</span><span style='background-color:rgb(129.07910406589508,125.92089593410492,125.92089593410492)'> allan</span><span style='background-color:rgb(123.67196381092072,131.32803618907928,131.32803618907928)'> thought</span><span style='background-color:rgb(125.12926161289215,129.87073838710785,129.87073838710785)'> it</span><span style='background-color:rgb(127.42018908262253,127.57981091737747,127.57981091737747)'> had</span><span style='background-color:rgb(138.38059455156326,116.61940544843674,116.61940544843674)'> potential</span><span style='background-color:rgb(140.4421891272068,114.5578108727932,114.5578108727932)'> .</span><span style='background-color:rgb(130.11476129293442,124.88523870706558,124.88523870706558)'> besides</span><span style='background-color:rgb(129.99335780739784,125.00664219260216,125.00664219260216)'> ,</span><span style='background-color:rgb(138.73054057359695,116.26945942640305,116.26945942640305)'> the</span><span style='background-color:rgb(132.44841367006302,122.55158632993698,122.55158632993698)'> public</span><span style='background-color:rgb(137.6709370315075,117.32906296849251,117.32906296849251)'> liked</span><span style='background-color:rgb(134.76598531007767,120.23401468992233,120.23401468992233)'> a</span><span style='background-color:rgb(128.74347567558289,126.25652432441711,126.25652432441711)'> good</span><span style='background-color:rgb(131.79861053824425,123.20138946175575,123.20138946175575)'> ,</span><span style='background-color:rgb(143.91147136688232,111.08852863311768,111.08852863311768)'> simple</span><span style='background-color:rgb(143.83301317691803,111.16698682308197,111.16698682308197)'> romance</span><span style='background-color:rgb(124.36565458774567,130.63434541225433,130.63434541225433)'> .</span><span style='background-color:rgb(132.82302036881447,122.17697963118553,122.17697963118553)'> perhaps</span><span style='background-color:rgb(142.2997422516346,112.7002577483654,112.7002577483654)'> this</span><span style='background-color:rgb(129.62633550167084,125.37366449832916,125.37366449832916)'> will</span><span style='background-color:rgb(133.4459513425827,121.5540486574173,121.5540486574173)'> be</span><span style='background-color:rgb(133.47505018115044,121.52494981884956,121.52494981884956)'> more</span><span style='background-color:rgb(138.1804744899273,116.81952551007271,116.81952551007271)'> sale</span><span style='background-color:rgb(132.8973975777626,122.1026024222374,122.1026024222374)'> ##able</span><span style='background-color:rgb(132.0041263103485,122.99587368965149,122.99587368965149)'> ,</span><span style='background-color:rgb(132.22694635391235,122.77305364608765,122.77305364608765)'> he</span><span style='background-color:rgb(130.74608221650124,124.25391778349876,124.25391778349876)'> thought</span><span style='background-color:rgb(125.75524002313614,129.24475997686386,129.24475997686386)'> and</span><span style='background-color:rgb(138.88747975230217,116.11252024769783,116.11252024769783)'> began</span><span style='background-color:rgb(132.26346239447594,122.73653760552406,122.73653760552406)'> to</span><span style='background-color:rgb(136.53392404317856,118.46607595682144,118.46607595682144)'> write</span><span style='background-color:rgb(126.23921245336533,128.76078754663467,128.76078754663467)'> .</span><span style='background-color:rgb(128.23273748159409,126.76726251840591,126.76726251840591)'> they</span><span style='background-color:rgb(132.40609154105186,122.59390845894814,122.59390845894814)'> gazed</span><span style='background-color:rgb(132.83840954303741,122.16159045696259,122.16159045696259)'> at</span><span style='background-color:rgb(127.86185458302498,127.13814541697502,127.13814541697502)'> each</span><span style='background-color:rgb(137.89215356111526,117.10784643888474,117.10784643888474)'> other</span><span style='background-color:rgb(142.4053005874157,112.5946994125843,112.5946994125843)'> ,</span><span style='background-color:rgb(134.0552182495594,120.9447817504406,120.9447817504406)'> lost</span><span style='background-color:rgb(122.54102289676666,132.45897710323334,132.45897710323334)'> in</span><span style='background-color:rgb(138.7263683974743,116.27363160252571,116.27363160252571)'> the</span><span style='background-color:rgb(134.63708102703094,120.36291897296906,120.36291897296906)'> rap</span><span style='background-color:rgb(129.68948811292648,125.31051188707352,125.31051188707352)'> ##ture</span><span style='background-color:rgb(132.3191674053669,122.6808325946331,122.6808325946331)'> of</span><span style='background-color:rgb(139.59080681204796,115.40919318795204,115.40919318795204)'> love</span><span style='background-color:rgb(130.3494442999363,124.6505557000637,124.6505557000637)'> based</span><span style='background-color:rgb(130.37161231040955,124.62838768959045,124.62838768959045)'> so</span><span style='background-color:rgb(137.1583293378353,117.84167066216469,117.84167066216469)'> deeply</span><span style='background-color:rgb(138.87294933199883,116.12705066800117,116.12705066800117)'> within</span><span style='background-color:rgb(140.15841275453568,114.84158724546432,114.84158724546432)'> their</span><span style='background-color:rgb(133.50664168596268,121.49335831403732,121.49335831403732)'> hearts</span><span style='background-color:rgb(124.71351832151413,130.28648167848587,130.28648167848587)'> that</span><span style='background-color:rgb(135.59880942106247,119.40119057893753,119.40119057893753)'> they</span><span style='background-color:rgb(139.29450631141663,115.70549368858337,115.70549368858337)'> had</span><span style='background-color:rgb(135.75622737407684,119.24377262592316,119.24377262592316)'> never</span><span style='background-color:rgb(131.81415930390358,123.18584069609642,123.18584069609642)'> seen</span><span style='background-color:rgb(131.13539412617683,123.86460587382317,123.86460587382317)'> it</span><span style='background-color:rgb(133.35477143526077,121.64522856473923,121.64522856473923)'> before</span><span style='background-color:rgb(146.41305953264236,108.58694046735764,108.58694046735764)'> .</span><span style='background-color:rgb(130.67922860383987,124.32077139616013,124.32077139616013)'> "</span><span style='background-color:rgb(118.18852365016937,136.81147634983063,136.81147634983063)'> what</span><span style='background-color:rgb(142.50476405024529,112.49523594975471,112.49523594975471)'> about</span><span style='background-color:rgb(145.48425257205963,109.51574742794037,109.51574742794037)'> roger</span><span style='background-color:rgb(135.4119050502777,119.58809494972229,119.58809494972229)'> ?</span><span style='background-color:rgb(143.11068713665009,111.88931286334991,111.88931286334991)'> "</span><span style='background-color:rgb(121.95623427629471,133.0437657237053,133.0437657237053)'> she</span><span style='background-color:rgb(144.03162091970444,110.96837908029556,110.96837908029556)'> asked</span><span style='background-color:rgb(154.84071850776672,100.15928149223328,100.15928149223328)'> ,</span><span style='background-color:rgb(150.9725260734558,104.02747392654419,104.02747392654419)'> knowing</span><span style='background-color:rgb(125.64952969551086,129.35047030448914,129.35047030448914)'> that</span><span style='background-color:rgb(159.3455895781517,95.6544104218483,95.6544104218483)'> the</span><span style='background-color:rgb(133.43104854226112,121.56895145773888,121.56895145773888)'> answer</span><span style='background-color:rgb(145.9639160335064,109.0360839664936,109.0360839664936)'> no</span><span style='background-color:rgb(117.72753238677979,137.27246761322021,137.27246761322021)'> longer</span><span style='background-color:rgb(117.43216663599014,137.56783336400986,137.56783336400986)'> mattered</span><span style='background-color:rgb(111.52178138494492,143.47821861505508,143.47821861505508)'> .</span><span style='background-color:rgb(122.26651042699814,132.73348957300186,132.73348957300186)'> [SEP]</span><span style='background-color:rgb(123.1787958741188,131.8212041258812,131.8212041258812)'> name</span><span style='background-color:rgb(75.37806004285812,179.62193995714188,179.62193995714188)'> few</span><span style='background-color:rgb(147.19997450709343,107.80002549290657,107.80002549290657)'> objects</span><span style='background-color:rgb(143.42519626021385,111.57480373978615,111.57480373978615)'> said</span><span style='background-color:rgb(135.34220159053802,119.65779840946198,119.65779840946198)'> to</span><span style='background-color:rgb(133.0127213895321,121.98727861046791,121.98727861046791)'> be</span><span style='background-color:rgb(114.97486889362335,140.02513110637665,140.02513110637665)'> in</span><span style='background-color:rgb(148.5510604083538,106.4489395916462,106.4489395916462)'> or</span><span style='background-color:rgb(133.619776815176,121.38022318482399,121.38022318482399)'> on</span><span style='background-color:rgb(136.02782920002937,118.97217079997063,118.97217079997063)'> allan</span><span style='background-color:rgb(155.26934310793877,99.73065689206123,99.73065689206123)'> '</span><span style='background-color:rgb(177.60538056492805,77.39461943507195,77.39461943507195)'> s</span><span style='background-color:rgb(98.37439566850662,156.62560433149338,156.62560433149338)'> desk</span><span style='background-color:rgb(155.05663812160492,99.94336187839508,99.94336187839508)'> |</span><span style='background-color:rgb(93.02367478609085,161.97632521390915,161.97632521390915)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> erase</span> ##r<span style='background-color:rgb(231.040917634964,23.95908236503601,23.95908236503601)'> [SEP]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span>

In [87]:
j = 0
grouped_attrs_intgrad = []
for i in tqdm(range(2)):
    intgrad_expln = intgrad_explainer(inputs[j][None], torch.tensor([i]))
    print('intgrad_expln.attributions', intgrad_expln.attributions.shape)
    grouped_attrs_intgrad.append(intgrad_expln.attributions)
print('torch.stack(grouped_attrs_intgrad)', torch.cat(grouped_attrs_intgrad).shape)
grouped_attrs_intgrad_aggr = torch.cat(grouped_attrs_intgrad).sum(-1)
grouped_attrs_intgrad_aggr, torch.argmax(grouped_attrs_intgrad_aggr)

  0%|          | 0/2 [00:00<?, ?it/s]

intgrad_expln.attributions torch.Size([1, 512])
intgrad_expln.attributions torch.Size([1, 512])
torch.stack(grouped_attrs_intgrad) torch.Size([2, 512])


(tensor([[-1.7239e-07],
         [-4.0127e-07]], device='cuda:0'),
 tensor(0, device='cuda:0'))

## GradCAM

In [28]:
!pip uninstall -y grad-cam

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Found existing installation: grad-cam 1.4.8
Uninstalling grad-cam-1.4.8:
  Successfully uninstalled grad-cam-1.4.8


In [16]:

import sys
sys.path.insert(0, '/shared_data0/weiqiuy/github/pytorch-grad-cam')
import pytorch_grad_cam
pytorch_grad_cam.__file__

'/shared_data0/weiqiuy/github/pytorch-grad-cam/pytorch_grad_cam/__init__.py'

In [74]:
import numpy as np
import torch
import ttach as tta
from typing import Callable, List, Tuple

from pytorch_grad_cam.activations_and_gradients import ActivationsAndGradients
from pytorch_grad_cam.utils.svd_on_activations import get_2d_projection
from pytorch_grad_cam.utils.image import scale_cam_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

class BaseCAMText:
    def __init__(self,
                 model: torch.nn.Module,
                 target_layers: List[torch.nn.Module],
                 use_cuda: bool = False,
                 reshape_transform: Callable = None,
                 compute_input_gradient: bool = False,
                 uses_gradients: bool = True) -> None:
        self.model = model.eval()
        self.target_layers = target_layers
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()
        self.reshape_transform = reshape_transform
        self.compute_input_gradient = compute_input_gradient
        self.uses_gradients = uses_gradients
        self.activations_and_grads = ActivationsAndGradients(
            self.model, target_layers, reshape_transform)

    """ Get a vector of weights for every channel in the target layer.
        Methods that return weights channels,
        will typically need to only implement this function. """

    def get_cam_weights(self,
                        input_tensor: torch.Tensor,
                        target_layers: List[torch.nn.Module],
                        targets: List[torch.nn.Module],
                        activations: torch.Tensor,
                        grads: torch.Tensor) -> np.ndarray:
        raise Exception("Not Implemented")

    def get_cam_image(self,
                      input_tensor: torch.Tensor,
                      target_layer: torch.nn.Module,
                      targets: List[torch.nn.Module],
                      activations: torch.Tensor,
                      grads: torch.Tensor,
                      eigen_smooth: bool = False) -> np.ndarray:

        weights = self.get_cam_weights(input_tensor,
                                       target_layer,
                                       targets,
                                       activations,
                                       grads)
        import pdb; pdb.set_trace()
        if len(activations.shape) == 3:
            weighted_activations = weights[:, :, None] * activations
        elif len(activations.shape) == 2:
            weighted_activations = weights[:, None] * activations
        else:
            raise Exception("Unsupported shape for activations: " + activations.shape)
        # cam = weighted_activations.sum(axis=-1)
        # if eigen_smooth:
        #     cam = get_2d_projection(weighted_activations)
        # else:
        #     cam = weighted_activations.sum(axis=1)
        cam = weighted_activations[:,0]
        return cam

    def forward(self,
                input_tensor: torch.Tensor,
                targets: List[torch.nn.Module],
                eigen_smooth: bool = False) -> np.ndarray:

        if self.cuda:
            input_tensor = input_tensor.cuda()

        if self.compute_input_gradient:
            input_tensor = torch.autograd.Variable(input_tensor,
                                                   requires_grad=True)

        outputs = self.activations_and_grads(input_tensor)
        if targets is None:
            target_categories = np.argmax(outputs.cpu().data.numpy(), axis=-1)
            targets = [ClassifierOutputTarget(
                category) for category in target_categories]

        if self.uses_gradients:
            self.model.zero_grad()
            loss = sum([target(output)
                       for target, output in zip(targets, outputs)])
            loss.backward(retain_graph=True)

        # In most of the saliency attribution papers, the saliency is
        # computed with a single target layer.
        # Commonly it is the last convolutional layer.
        # Here we support passing a list with multiple target layers.
        # It will compute the saliency image for every image,
        # and then aggregate them (with a default mean aggregation).
        # This gives you more flexibility in case you just want to
        # use all conv layers for example, all Batchnorm layers,
        # or something else.
        cam_per_layer = self.compute_cam_per_layer(input_tensor,
                                                   targets,
                                                   eigen_smooth)
        return self.aggregate_multi_layers(cam_per_layer)

    def get_target_width_height(self,
                                input_tensor: torch.Tensor) -> Tuple[int, int]:
        width, height = input_tensor.size(-1), input_tensor.size(-2)
        return width, height

    def compute_cam_per_layer(
            self,
            input_tensor: torch.Tensor,
            targets: List[torch.nn.Module],
            eigen_smooth: bool) -> np.ndarray:
        activations_list = [a.cpu().data.numpy()
                            for a in self.activations_and_grads.activations]
        grads_list = [g.cpu().data.numpy()
                      for g in self.activations_and_grads.gradients]
        target_size = self.get_target_width_height(input_tensor)

        cam_per_target_layer = []
        # Loop over the saliency image from every layer
        for i in range(len(self.target_layers)):
            target_layer = self.target_layers[i]
            layer_activations = None
            layer_grads = None
            if i < len(activations_list):
                layer_activations = activations_list[i]
            if i < len(grads_list):
                layer_grads = grads_list[i]

            cam = self.get_cam_image(input_tensor,
                                     target_layer,
                                     targets,
                                     layer_activations,
                                     layer_grads,
                                     eigen_smooth)
            cam = np.maximum(cam, 0)
            scaled = scale_cam_image(cam, target_size)
            cam_per_target_layer.append(scaled[:, None, :])

        return cam_per_target_layer

    def aggregate_multi_layers(
            self,
            cam_per_target_layer: np.ndarray) -> np.ndarray:
        cam_per_target_layer = np.concatenate(cam_per_target_layer, axis=1)
        cam_per_target_layer = np.maximum(cam_per_target_layer, 0)
        result = np.mean(cam_per_target_layer, axis=1)
        return scale_cam_image(result)

    def forward_augmentation_smoothing(self,
                                       input_tensor: torch.Tensor,
                                       targets: List[torch.nn.Module],
                                       eigen_smooth: bool = False) -> np.ndarray:
        transforms = tta.Compose(
            [
                tta.HorizontalFlip(),
                tta.Multiply(factors=[0.9, 1, 1.1]),
            ]
        )
        cams = []
        for transform in transforms:
            augmented_tensor = transform.augment_image(input_tensor)
            cam = self.forward(augmented_tensor,
                               targets,
                               eigen_smooth)

            # The ttach library expects a tensor of size BxCxHxW
            cam = cam[:, None, :, :]
            cam = torch.from_numpy(cam)
            cam = transform.deaugment_mask(cam)

            # Back to numpy float32, HxW
            cam = cam.numpy()
            cam = cam[:, 0, :, :]
            cams.append(cam)

        cam = np.mean(np.float32(cams), axis=0)
        return cam

    def __call__(self,
                 input_tensor: torch.Tensor,
                 targets: List[torch.nn.Module] = None,
                 aug_smooth: bool = False,
                 eigen_smooth: bool = False) -> np.ndarray:

        # Smooth the CAM result with test time augmentation
        if aug_smooth is True:
            return self.forward_augmentation_smoothing(
                input_tensor, targets, eigen_smooth)

        return self.forward(input_tensor,
                            targets, eigen_smooth)

    def __del__(self):
        self.activations_and_grads.release()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self.activations_and_grads.release()
        if isinstance(exc_value, IndexError):
            # Handle IndexError here...
            print(
                f"An exception occurred in CAM with block: {exc_type}. Message: {exc_value}")
            return True


In [75]:
from exlib.explainers.common import *
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget, ClassifierOutputSoftmaxTarget

class WrappedModelGradCAM(torch.nn.Module):
    def __init__(self, model): 
        super(WrappedModelGradCAM, self).__init__()
        self.model = model
        for name, param in self.model.named_parameters():
            param.requires_grad = True
            
    def forward(self, x):
        return self.model(x)

class GradCAMCls(FeatureAttrMethod):
    def __init__(self, model, target_layers, reshape_transform=None):
        
        model = WrappedModelGradCAM(model)

        super().__init__(model)
        
        self.target_layers = target_layers
        with torch.enable_grad():
            self.grad_cam = GradCAM(model=model, target_layers=self.target_layers,
                                    reshape_transform=reshape_transform,
                                    use_cuda=True if torch.cuda.is_available() else False)

    def forward(self, X, label=None, target_func=ClassifierOutputSoftmaxTarget):
        with torch.enable_grad():
            grad_cam_result = self.grad_cam(input_tensor=X, targets=[target_func(label) for label in label])
            grad_cam_result = torch.tensor(grad_cam_result)

        return FeatureAttrOutput(grad_cam_result.unsqueeze(1), grad_cam_result)

    
class GradCAMText(BaseCAMText):
    def __init__(self, model, target_layers, use_cuda=False,
                 reshape_transform=None):
        super(
            GradCAMText,
            self).__init__(
            model,
            target_layers,
            use_cuda,
            reshape_transform)

    def get_cam_weights(self,
                        input_tensor,
                        target_layer,
                        target_category,
                        activations,
                        grads):
        return np.mean(grads, axis=(-1))

class GradCAMTextCls(FeatureAttrMethod):
    def __init__(self, model, target_layers, reshape_transform=None):
        model = WrappedModelGradCAM(model)

        super().__init__(model)
        
        self.target_layers = target_layers
        with torch.enable_grad():
            self.grad_cam = GradCAMText(model=model, target_layers=self.target_layers,
                                    reshape_transform=reshape_transform,
                                    use_cuda=True if torch.cuda.is_available() else False)
 
    def forward(self, X, label=None, target_func=ClassifierOutputSoftmaxTarget):
        grad_cam_results = []
        for i in range(len(label)):
            with torch.enable_grad():
                grad_cam_result = self.grad_cam(input_tensor=X[i:i+1], 
                                                targets=[target_func(label[i:i+1])])
                # import pdb; pdb.set_trace()
                grad_cam_result = torch.tensor(grad_cam_result)
                grad_cam_results.append(grad_cam_result)
        grad_cam_results = torch.cat(grad_cam_results)

        return FeatureAttrOutput(grad_cam_results.squeeze(1), {})

In [76]:
gradcam_explainer = GradCAMTextCls(original_model2, 
                                    [original_model2.model.bert.encoder.layer[-1].output.LayerNorm]).to(device)

gradcam_expln = gradcam_explainer(inputs, labels)

> /tmp/ipykernel_3796/2149128867.py(56)get_cam_image()
     54                                        grads)
     55         import pdb; pdb.set_trace()
---> 56         if len(activations.shape) == 3:
     57             weighted_activations = weights[:, :, None] * activations
     58         elif len(activations.shape) == 2:



ipdb>  c


> /tmp/ipykernel_3796/2149128867.py(56)get_cam_image()
     54                                        grads)
     55         import pdb; pdb.set_trace()
---> 56         if len(activations.shape) == 3:
     57             weighted_activations = weights[:, :, None] * activations
     58         elif len(activations.shape) == 2:



ipdb>  c


In [77]:
gradcam_expln.attributions.shape

torch.Size([2, 512])

In [78]:
gradcam_expln.attributions, gradcam_expln.attributions.sum()

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor(0.))

In [47]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks, normalize=True):
    if normalize:
        masks_max = masks.max(-1).values
        masks_min = masks.min(-1).values
        # import pdb; pdb.set_trace()
        masks = (masks - masks_min) / (masks_max - masks_min)
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

tokens = processor.convert_ids_to_tokens(inputs[0])
# tokens = shap_expln.explainer_output.data[0]
printmd(show_attrs(tokens, gradcam_expln.attributions[0]))

<span style='background-color:rgb(nan,nan,nan)'> [CLS]</span><span style='background-color:rgb(nan,nan,nan)'> allan</span><span style='background-color:rgb(nan,nan,nan)'> sat</span><span style='background-color:rgb(nan,nan,nan)'> down</span><span style='background-color:rgb(nan,nan,nan)'> at</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> desk</span><span style='background-color:rgb(nan,nan,nan)'> and</span><span style='background-color:rgb(nan,nan,nan)'> pulled</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> chair</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> close</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> opening</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> side</span><span style='background-color:rgb(nan,nan,nan)'> drawer</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> he</span><span style='background-color:rgb(nan,nan,nan)'> took</span><span style='background-color:rgb(nan,nan,nan)'> out</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> piece</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> paper</span><span style='background-color:rgb(nan,nan,nan)'> and</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> ink</span><span style='background-color:rgb(nan,nan,nan)'> ##pot</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> after</span><span style='background-color:rgb(nan,nan,nan)'> filling</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> pen</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> allan</span><span style='background-color:rgb(nan,nan,nan)'> looked</span><span style='background-color:rgb(nan,nan,nan)'> at</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> paper</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> orange</span><span style='background-color:rgb(nan,nan,nan)'> glow</span><span style='background-color:rgb(nan,nan,nan)'> from</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> lantern</span><span style='background-color:rgb(nan,nan,nan)'> set</span><span style='background-color:rgb(nan,nan,nan)'> back</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> desk</span><span style='background-color:rgb(nan,nan,nan)'> '</span><span style='background-color:rgb(nan,nan,nan)'> s</span><span style='background-color:rgb(nan,nan,nan)'> right</span><span style='background-color:rgb(nan,nan,nan)'> -</span><span style='background-color:rgb(nan,nan,nan)'> hand</span><span style='background-color:rgb(nan,nan,nan)'> corner</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> pen</span><span style='background-color:rgb(nan,nan,nan)'> cast</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> forbid</span><span style='background-color:rgb(nan,nan,nan)'> ##ding</span><span style='background-color:rgb(nan,nan,nan)'> line</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> shadow</span><span style='background-color:rgb(nan,nan,nan)'> slant</span><span style='background-color:rgb(nan,nan,nan)'> ##ing</span><span style='background-color:rgb(nan,nan,nan)'> across</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> page</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> echoing</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> ink</span><span style='background-color:rgb(nan,nan,nan)'> ##y</span><span style='background-color:rgb(nan,nan,nan)'> darkness</span><span style='background-color:rgb(nan,nan,nan)'> crouch</span><span style='background-color:rgb(nan,nan,nan)'> ##ing</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> edges</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> lantern</span><span style='background-color:rgb(nan,nan,nan)'> '</span><span style='background-color:rgb(nan,nan,nan)'> s</span><span style='background-color:rgb(nan,nan,nan)'> struggling</span><span style='background-color:rgb(nan,nan,nan)'> glow</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> only</span><span style='background-color:rgb(nan,nan,nan)'> other</span><span style='background-color:rgb(nan,nan,nan)'> illumination</span><span style='background-color:rgb(nan,nan,nan)'> came</span><span style='background-color:rgb(nan,nan,nan)'> from</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> lu</span><span style='background-color:rgb(nan,nan,nan)'> ##rid</span><span style='background-color:rgb(nan,nan,nan)'> moonlight</span><span style='background-color:rgb(nan,nan,nan)'> filtered</span><span style='background-color:rgb(nan,nan,nan)'> through</span><span style='background-color:rgb(nan,nan,nan)'> thin</span><span style='background-color:rgb(nan,nan,nan)'> branches</span><span style='background-color:rgb(nan,nan,nan)'> and</span><span style='background-color:rgb(nan,nan,nan)'> clouds</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> casting</span><span style='background-color:rgb(nan,nan,nan)'> its</span><span style='background-color:rgb(nan,nan,nan)'> bone</span><span style='background-color:rgb(nan,nan,nan)'> -</span><span style='background-color:rgb(nan,nan,nan)'> pale</span><span style='background-color:rgb(nan,nan,nan)'> glow</span><span style='background-color:rgb(nan,nan,nan)'> onto</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> pine</span><span style='background-color:rgb(nan,nan,nan)'> floor</span><span style='background-color:rgb(nan,nan,nan)'> ##boards</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> allan</span><span style='background-color:rgb(nan,nan,nan)'> unfolded</span><span style='background-color:rgb(nan,nan,nan)'> another</span><span style='background-color:rgb(nan,nan,nan)'> page</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> this</span><span style='background-color:rgb(nan,nan,nan)'> one</span><span style='background-color:rgb(nan,nan,nan)'> crowded</span><span style='background-color:rgb(nan,nan,nan)'> with</span><span style='background-color:rgb(nan,nan,nan)'> ranks</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> letters</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> tight</span><span style='background-color:rgb(nan,nan,nan)'> formation</span><span style='background-color:rgb(nan,nan,nan)'> from</span><span style='background-color:rgb(nan,nan,nan)'> left</span><span style='background-color:rgb(nan,nan,nan)'> to</span><span style='background-color:rgb(nan,nan,nan)'> right</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> lines</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> letters</span><span style='background-color:rgb(nan,nan,nan)'> stepped</span><span style='background-color:rgb(nan,nan,nan)'> into</span><span style='background-color:rgb(nan,nan,nan)'> their</span><span style='background-color:rgb(nan,nan,nan)'> divisions</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> shape</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> story</span><span style='background-color:rgb(nan,nan,nan)'> '</span><span style='background-color:rgb(nan,nan,nan)'> s</span><span style='background-color:rgb(nan,nan,nan)'> outline</span><span style='background-color:rgb(nan,nan,nan)'> :</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> loose</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> dry</span><span style='background-color:rgb(nan,nan,nan)'> skeleton</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> tale</span><span style='background-color:rgb(nan,nan,nan)'> lay</span><span style='background-color:rgb(nan,nan,nan)'> exposed</span><span style='background-color:rgb(nan,nan,nan)'> beneath</span><span style='background-color:rgb(nan,nan,nan)'> their</span><span style='background-color:rgb(nan,nan,nan)'> feet</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> awaiting</span><span style='background-color:rgb(nan,nan,nan)'> tend</span><span style='background-color:rgb(nan,nan,nan)'> ##ons</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> muscle</span><span style='background-color:rgb(nan,nan,nan)'> and</span><span style='background-color:rgb(nan,nan,nan)'> blushing</span><span style='background-color:rgb(nan,nan,nan)'> skin</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> allan</span><span style='background-color:rgb(nan,nan,nan)'> reviewed</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> troops</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> all</span><span style='background-color:rgb(nan,nan,nan)'> prepared</span><span style='background-color:rgb(nan,nan,nan)'> to</span><span style='background-color:rgb(nan,nan,nan)'> di</span><span style='background-color:rgb(nan,nan,nan)'> ##se</span><span style='background-color:rgb(nan,nan,nan)'> ##mba</span><span style='background-color:rgb(nan,nan,nan)'> ##rk</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> their</span><span style='background-color:rgb(nan,nan,nan)'> task</span><span style='background-color:rgb(nan,nan,nan)'> to</span><span style='background-color:rgb(nan,nan,nan)'> form</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> tale</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> young</span><span style='background-color:rgb(nan,nan,nan)'> man</span><span style='background-color:rgb(nan,nan,nan)'> returning</span><span style='background-color:rgb(nan,nan,nan)'> home</span><span style='background-color:rgb(nan,nan,nan)'> from</span><span style='background-color:rgb(nan,nan,nan)'> life</span><span style='background-color:rgb(nan,nan,nan)'> abroad</span><span style='background-color:rgb(nan,nan,nan)'> to</span><span style='background-color:rgb(nan,nan,nan)'> find</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> childhood</span><span style='background-color:rgb(nan,nan,nan)'> friend</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> bride</span><span style='background-color:rgb(nan,nan,nan)'> to</span><span style='background-color:rgb(nan,nan,nan)'> -</span><span style='background-color:rgb(nan,nan,nan)'> be</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> thus</span><span style='background-color:rgb(nan,nan,nan)'> upset</span><span style='background-color:rgb(nan,nan,nan)'> ##ting</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> apple</span><span style='background-color:rgb(nan,nan,nan)'> cart</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> life</span><span style='background-color:rgb(nan,nan,nan)'> '</span><span style='background-color:rgb(nan,nan,nan)'> s</span><span style='background-color:rgb(nan,nan,nan)'> plan</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> clarified</span><span style='background-color:rgb(nan,nan,nan)'> –</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> course</span><span style='background-color:rgb(nan,nan,nan)'> –</span><span style='background-color:rgb(nan,nan,nan)'> by</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> very</span><span style='background-color:rgb(nan,nan,nan)'> time</span><span style='background-color:rgb(nan,nan,nan)'> away</span><span style='background-color:rgb(nan,nan,nan)'> from</span><span style='background-color:rgb(nan,nan,nan)'> her</span><span style='background-color:rgb(nan,nan,nan)'> he</span><span style='background-color:rgb(nan,nan,nan)'> loved</span><span style='background-color:rgb(nan,nan,nan)'> best</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> although</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> concept</span><span style='background-color:rgb(nan,nan,nan)'> was</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> simple</span><span style='background-color:rgb(nan,nan,nan)'> one</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> allan</span><span style='background-color:rgb(nan,nan,nan)'> thought</span><span style='background-color:rgb(nan,nan,nan)'> it</span><span style='background-color:rgb(nan,nan,nan)'> had</span><span style='background-color:rgb(nan,nan,nan)'> potential</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> besides</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> public</span><span style='background-color:rgb(nan,nan,nan)'> liked</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> good</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> simple</span><span style='background-color:rgb(nan,nan,nan)'> romance</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> perhaps</span><span style='background-color:rgb(nan,nan,nan)'> this</span><span style='background-color:rgb(nan,nan,nan)'> will</span><span style='background-color:rgb(nan,nan,nan)'> be</span><span style='background-color:rgb(nan,nan,nan)'> more</span><span style='background-color:rgb(nan,nan,nan)'> sale</span><span style='background-color:rgb(nan,nan,nan)'> ##able</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> he</span><span style='background-color:rgb(nan,nan,nan)'> thought</span><span style='background-color:rgb(nan,nan,nan)'> and</span><span style='background-color:rgb(nan,nan,nan)'> began</span><span style='background-color:rgb(nan,nan,nan)'> to</span><span style='background-color:rgb(nan,nan,nan)'> write</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> they</span><span style='background-color:rgb(nan,nan,nan)'> gazed</span><span style='background-color:rgb(nan,nan,nan)'> at</span><span style='background-color:rgb(nan,nan,nan)'> each</span><span style='background-color:rgb(nan,nan,nan)'> other</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> lost</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> rap</span><span style='background-color:rgb(nan,nan,nan)'> ##ture</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> love</span><span style='background-color:rgb(nan,nan,nan)'> based</span><span style='background-color:rgb(nan,nan,nan)'> so</span><span style='background-color:rgb(nan,nan,nan)'> deeply</span><span style='background-color:rgb(nan,nan,nan)'> within</span><span style='background-color:rgb(nan,nan,nan)'> their</span><span style='background-color:rgb(nan,nan,nan)'> hearts</span><span style='background-color:rgb(nan,nan,nan)'> that</span><span style='background-color:rgb(nan,nan,nan)'> they</span><span style='background-color:rgb(nan,nan,nan)'> had</span><span style='background-color:rgb(nan,nan,nan)'> never</span><span style='background-color:rgb(nan,nan,nan)'> seen</span><span style='background-color:rgb(nan,nan,nan)'> it</span><span style='background-color:rgb(nan,nan,nan)'> before</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> "</span><span style='background-color:rgb(nan,nan,nan)'> what</span><span style='background-color:rgb(nan,nan,nan)'> about</span><span style='background-color:rgb(nan,nan,nan)'> roger</span><span style='background-color:rgb(nan,nan,nan)'> ?</span><span style='background-color:rgb(nan,nan,nan)'> "</span><span style='background-color:rgb(nan,nan,nan)'> she</span><span style='background-color:rgb(nan,nan,nan)'> asked</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> knowing</span><span style='background-color:rgb(nan,nan,nan)'> that</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> answer</span><span style='background-color:rgb(nan,nan,nan)'> no</span><span style='background-color:rgb(nan,nan,nan)'> longer</span><span style='background-color:rgb(nan,nan,nan)'> mattered</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> [SEP]</span><span style='background-color:rgb(nan,nan,nan)'> name</span><span style='background-color:rgb(nan,nan,nan)'> few</span><span style='background-color:rgb(nan,nan,nan)'> objects</span><span style='background-color:rgb(nan,nan,nan)'> said</span><span style='background-color:rgb(nan,nan,nan)'> to</span><span style='background-color:rgb(nan,nan,nan)'> be</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> or</span><span style='background-color:rgb(nan,nan,nan)'> on</span><span style='background-color:rgb(nan,nan,nan)'> allan</span><span style='background-color:rgb(nan,nan,nan)'> '</span><span style='background-color:rgb(nan,nan,nan)'> s</span><span style='background-color:rgb(nan,nan,nan)'> desk</span><span style='background-color:rgb(nan,nan,nan)'> |</span><span style='background-color:rgb(nan,nan,nan)'> |</span><span style='background-color:rgb(nan,nan,nan)'> erase</span><span style='background-color:rgb(nan,nan,nan)'> ##r</span><span style='background-color:rgb(nan,nan,nan)'> [SEP]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span>

In [39]:
j = 0
grouped_attrs_gradcam = []
for i in tqdm(range(2)):
    gradcam_expln = gradcam_explainer(inputs[j][None], torch.tensor([i]))
    grouped_attrs_gradcam.append(gradcam_expln.attributions.sum(1).view(-1))
grouped_attrs_gradcam_aggr = torch.stack(grouped_attrs_gradcam).sum(-1)
grouped_attrs_gradcam_aggr, torch.argmax(grouped_attrs_gradcam_aggr)

  0%|          | 0/2 [00:00<?, ?it/s]

(tensor([25.0217, 90.4046]), tensor(1))

## IDG

In [15]:
!pip install graphviz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 1.1 MB/s eta 0:00:00


In [19]:
!pip install captum==0.2.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 41.2 MB/s eta 0:00:00
  Attempting uninstall: captum
    Found existing installation: captum 0.7.0
    Uninstalling captum-0.7.0:
      Successfully uninstalled captum-0.7.0


In [15]:
from exlib.explainers.idg import IDGTextCls

explainer = IDGTextCls(original_model3, processor).to(device)

Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
size_average and reduce args will be deprecated, please use reduction='sum' instead.


In [16]:
sentence = processor.decode(inputs[0])
sentence = sentence.replace('[CLS]', '').replace('[SEP]', '').strip()

idg_expln = explainer([sentence], labels)

indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/cuda/Indexing.cu:1440.)
indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)


class distribution: tensor([[3.0071e-04, 9.9970e-01]], device='cuda:0')


dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).


In [20]:
j = 0
grouped_attrs_idg = []
for i in tqdm(range(2)):
    idg_expln = explainer([sentence], torch.tensor([i]))
    print(idg_expln.attributions.shape)
    grouped_attrs_idg.append(idg_expln.attributions.sum(1).view(-1))
grouped_attrs_idg_aggr = torch.stack(grouped_attrs_idg).sum(-1)
grouped_attrs_idg_aggr, torch.argmax(grouped_attrs_idg_aggr)

  0%|          | 0/2 [00:00<?, ?it/s]

wrong inference
class distribution: tensor([[3.0071e-04, 9.9970e-01]], device='cuda:0')
torch.Size([1, 21])
class distribution: tensor([[3.0071e-04, 9.9970e-01]], device='cuda:0')
torch.Size([1, 21])


(tensor([6.0792, 6.0790]), tensor(0))

### testing

In [27]:
import sys
sys.path.append('../../lib/LAL-Parser')
sys.path.append('../../lib/LAL-Parser/src_joint')
from src_joint import KM_parser
from src_joint.main import REVERSE_TOKEN_MAPPING

def torch_load(load_path):
    if KM_parser.use_cuda:
        return torch.load(load_path)
    else:
        return torch.load(load_path, map_location=lambda storage, location: storage)
    
model_path_base = '../../lib/LAL-Parser/best_parser.pt'
info = torch_load(model_path_base)

In [16]:
!pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
parser = KM_parser.ChartParser.from_spec(info['spec'], info['state_dict'])
parser.eval();

Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetModel: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/conda/envs/rapids/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [18]:
sentence = processor.decode(inputs[0])
sentence = sentence.replace('[CLS]', '').replace('[SEP]', '').strip()
sentence

'it s a lovely film with lovely performances by buy and accorsi'

In [21]:
import nltk
from nltk import word_tokenize, sent_tokenize

subbatch_sentences = [sentence]

tagged_sentences = [[(REVERSE_TOKEN_MAPPING.get(tag, tag), REVERSE_TOKEN_MAPPING.get(word, word)) \
                     for word, tag in nltk.pos_tag(word_tokenize(sentence))] for sentence in subbatch_sentences]
syntree, _ = parser.parse_batch(tagged_sentences)
trees = [syntree[0].convert().linearize()]
trees

/shared_data0/weiqiuy/sop/notebooks/text/../../lib/LAL-Parser/src_joint/KM_parser.py:1763: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  features_packed = features.masked_select(all_word_end_mask.to(DTYPE).unsqueeze(-1)).reshape(-1,
/shared_data0/weiqiuy/sop/notebooks/text/../../lib/LAL-Parser/src_joint/KM_parser.py:230: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/cuda/Indexing.cu:1440.)
  attn.data.masked_fill_(attn_mask, -float('inf'))
/shared_data0/weiqiuy/sop/notebooks/text/../../lib/LAL-Parser/src_joint/KM_parser.py:397: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  outputs = ou

['(S (NP (PRP it)) (VP (VBZ s) (NP (NP (DT a) (JJ lovely) (NN film)) (PP (IN with) (NP (NP (JJ lovely) (NNS performances)) (PP (IN by) (NP (NN buy) (CC and) (NN accorsi))))))))']

In [34]:
import sys
sys.path.append('../../lib/integrated-directional-gradients')
from IDG.utils import load_model
# from transformers import XLNetTokenizer, BertTokenizer
import torch
from IDG.calculate_gradients import execute_IDG
import click

# trees = ["(3 (2 (2 The) (2 Rock)) (4 (3 (2 is) (4 (2 destined) (2 (2 (2 (2 (2 to) (2 (2 be) (2 (2 the) (2 (2 21st) (2 (2 (2 Century) (2 's)) (2 (3 new) (2 (2 ``) (2 Conan)))))))) (2 '')) (2 and)) (3 (2 that) (3 (2 he) (3 (2 's) (3 (2 going) (3 (2 to) (4 (3 (2 make) (3 (3 (2 a) (3 splash)) (2 (2 even) (3 greater)))) (2 (2 than) (2 (2 (2 (2 (1 (2 Arnold) (2 Schwarzenegger)) (2 ,)) (2 (2 Jean-Claud) (2 (2 Van) (2 Damme)))) (2 or)) (2 (2 Steven) (2 Segal))))))))))))) (2 .)))"]
coalitions1, value_func1 = execute_IDG(trees, original_model3, processor, 0, './tmp/', True)

wrong inference
class distribution: tensor([[3.0071e-04, 9.9970e-01]], device='cuda:0')


In [35]:
print(coalitions1, value_func1)

[['it', 's', 'a', 'lovely', 'film', 'with', 'lovely1', 'performances', 'by', 'buy', 'and', 'accorsi'], ['it'], ['s', 'a', 'lovely', 'film', 'with', 'lovely1', 'performances', 'by', 'buy', 'and', 'accorsi'], ['s'], ['a', 'lovely', 'film', 'with', 'lovely1', 'performances', 'by', 'buy', 'and', 'accorsi'], ['a', 'lovely', 'film'], ['with', 'lovely1', 'performances', 'by', 'buy', 'and', 'accorsi'], ['a'], ['lovely'], ['film'], ['with'], ['lovely1', 'performances', 'by', 'buy', 'and', 'accorsi'], ['lovely1', 'performances'], ['by', 'buy', 'and', 'accorsi'], ['lovely1'], ['performances'], ['by'], ['buy', 'and', 'accorsi'], ['buy'], ['and'], ['accorsi']] [0.9999999806438096, 0.05822701815330838, 0.9409372529921779, 0.06246160507153362, 0.8775737472853845, 0.27392191566821, 0.6027534810112597, 0.07219692879984181, 0.12678563489955144, 0.07395147153547421, 0.0670975132655819, 0.5351780857755253, 0.16356625462045807, 0.37100766298159743, 0.09643775143531089, 0.06611925976852186, 0.05059548087152

In [36]:
coalitions0, value_func0 = execute_IDG(trees, original_model3, processor, 0, './tmp/', True)
coalitions1, value_func1 = execute_IDG(trees, original_model3, processor, 1, './tmp/', True)
sum(value_func0), sum(value_func1)

wrong inference
class distribution: tensor([[3.0071e-04, 9.9970e-01]], device='cuda:0')
class distribution: tensor([[3.0071e-04, 9.9970e-01]], device='cuda:0')


(6.079246797266737, 6.0790257047504435)

## PLS

In [17]:
from exlib.explainers.pls import PLSTextCls

explainer = PLSTextCls(backbone_model, processor)

<exlib.explainers.pls.PLSTextCls.__init__.<locals>.Args object at 0x7f2c140bf8b0>


In [18]:
pls_expln = explainer(inputs, labels, **kwargs)

divide by zero encountered in divide
divide by zero encountered in divide


In [19]:
pls_expln.attributions

tensor([[0.0000, 0.0000, 0.0000, 0.0003, 0.0000, 0.0000, 0.0000, 0.0003, 0.0000,
         0.0003, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       device='cuda:0', dtype=torch.float64)

In [20]:
pls_expln.explainer_output

{'expln_flat_masks': tensor([[0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.]],
        device='cuda:0'),
 'masks': tensor([[0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.]],
        device='cuda:0'),
 'mask_weights': tensor([0.0003], device='cuda:0', dtype=torch.float64),
 'explanation_strs': ['_ _ _ lovely _ _ _ performances _ buy _ _ _ _']}

In [21]:
j = 0
grouped_attrs = []
for i in tqdm(range(2)):
    expln_i = explainer(inputs, torch.tensor([i]).to(device), **kwargs)
    grouped_attrs.append(expln_i.explainer_output['mask_weights'])
grouped_attrs_aggr = torch.tensor(grouped_attrs) #torch.tensor([sum(ga) for ga in grouped_attrs]).to(device)
aggr_pred = torch.argmax(grouped_attrs_aggr)
aggr_pred, grouped_attrs_aggr

100%|██████████| 2/2 [00:00<00:00,  2.01it/s]


(tensor(1), tensor([-8.6170e-01,  2.5970e-04], dtype=torch.float64))